In [32]:
LOAD_LIBS = False
if LOAD_LIBS:
    !pip install -q -U catboost
    !pip install -q -U lightgbm
    !pip install -q -U xgboost
    !pip install -q -U geopy
    !pip install -q -U phik
    
    

In [33]:
import os
import gc
import json
import time
import logging
import argparse
from tqdm import tqdm
from typing import List, Dict, Tuple, Optional
from transformers import AutoTokenizer, AutoModel, CLIPModel, CLIPProcessor


import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import (
    roc_auc_score,
    accuracy_score,
    log_loss,
    mean_squared_error,
    mean_absolute_error,
    confusion_matrix
)

from catboost import CatBoostClassifier, CatBoostRegressor, Pool
import lightgbm as lgb
import xgboost as xgb

from PIL import Image

import torch
import torch.nn as nn
from torchvision import models, transforms
from torch.utils.data import DataLoader

from transformers import AutoTokenizer, AutoModel
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
# Гео-бэкенд: geopy (если установлен) или haversine
try:
    from geopy.distance import geodesic
    GEO_BACKEND = "geopy"
except ImportError:
    GEO_BACKEND = "haversine"


try:
    import phik
    HAS_PHIK = True
except ImportError:
    HAS_PHIK = False

In [34]:
GEO_BACKEND, HAS_PHIK

('geopy', True)

In [35]:
# для текста, никак не трогать
TEXT_MODEL = None
TEXT_TOKENIZER = None
TEXT_DEVICE = None

# TF-IDF и SVD отдельно для каждого текстового поля (чтобы не мешать разные источники)
TFIDF_VECTORIZERS = {}  # field_name -> vectorizer
TFIDF_SVDS = {}         # field_name -> svd


In [ ]:
# DATA_FOLDER_PATH = '.'
# CONFIG = {
#     # ---------- Paths ----------
#     "train_path": os.path.join(DATA_FOLDER_PATH, "/kaggle/input/petfinder-pawpularity-score/train.csv"),
#     "test_path": os.path.join(DATA_FOLDER_PATH, '/kaggle/input/petfinder-pawpularity-score/test.csv'),
#     "sep": ",",
#     "id_column": "Id",
#     "target_column": "Pawpularity",
#     "datetime_columns": [],
#     "output_dir": "tabular_boosting_output",

#     # ---------- Task ----------
#     # "binary" / "multiclass" / "regression"
#     "task_type": "regression",

#     # ---------- Basic features ----------
#     "basic_drop_columns": [],
#     "basic_as_categorical": [],
#     "basic_max_cat_unique": 64,
#     "basic_datetime_expand": False,
#     "basic_datetime_features": ["year", "month", "day", "dow", "hour"],

#     # ---------- Categorical processing ----------
#     # минимальный "размер" категории (по числу объектов и/или доле),
#     # все категории меньше порога сливаются в 'other'
#     "cat_min_count": 20,          # минимальное кол-во объектов в категории
#     "cat_min_freq": 0.0,          # минимальная доля (0..1); 0 = игнорировать
#     # если доля "редких" категорий в признаке >= этого порога — признак удаляется
#     "cat_max_rare_share": 0.98,
#     # если после объединения в other уникальных всё ещё слишком много — признак удаляется
#     "cat_max_unique_after_group": 500,

#     # ---------- Post-feature service columns ----------
#     # служебные колонки, которые нужно удалить после генерации фич
#     # (исходные координаты, raw id, сырые адреса и т.п.)
#     "post_feature_drop_columns": [], # может быть удалить category_geo_ref_lat, category_geo_ref_lon

#     # ---------- Address processing ----------
#     # колонка-адрес, из которой нужно извлечь город
#     "address_column": None,  

#     # индекс элемента после split (например [-1] = последний)
#     # можно указывать как отрицательный индекс
#     "address_city_index": -1,  

#     # разделитель в строке адреса
#     "address_split_sep": ",",  

#     # ---------- Aggregates ----------
#     "agg_enable": False,
#     "agg_groupby_cols": [],
#     "agg_numeric_cols": [],
#     "agg_aggs": ["mean", "std", "min", "max", "sum", "median", "nunique", "count"],
#     "agg_prefix": "agg",

#     # ---------- Geo features ----------
#     "geo_enable": False, 
#     "geo_lat_from_col": "pickup_lat",
#     "geo_lon_from_col": "pickup_lon",
#     "geo_lat_to_col": "dropoff_lat",
#     "geo_lon_to_col": "dropoff_lon",
#     "geo_ref_lat": 'category_geo_ref_lat',
#     "geo_ref_lon": 'category_geo_ref_lon',
#     "geo_prefix": "geo",
#     # дополнительные фичи (bearing, manhattan, dlat/dlon, midpoint)
#     "geo_extra_enable": False,

#     "geo_from_coord_col": None,
#     "geo_to_coord_col": None,
#     "geo_coord_string_sep": ",",

#     # ---------- Geo reference config ----------
#     "geo_reference": {
#         "enabled": False,
#         "category_column": "category",
#         "coordinates_column": "coordinates",  # строка "lat,lon", если lat/lon не заданы
#         "lat_column": None,                  # <- если задать, берём отсюда широту
#         "lon_column": None,                  # <- если задать, берём отсюда долготу
#         "output_column": "category_geo_ref",
#         "output_lat_column": "category_geo_ref_lat",
#         "output_lon_column": "category_geo_ref_lon",
#     },

#     # ---------- Correlation-based feature filtering ----------
#     "corr_enable": False,          # включить/выключить фильтрацию по корреляции
#     "corr_pearson_min_abs": 0.95,   # порог |Pearson|; 0.1, 0.05 и т.п.
#     "corr_use_phik": False,        # считать ли phik
#     "corr_phik_min_abs": 0.0,      # порог |phik|
    
#     #----------images---------
#     # папка с train-картинками (если используем image-фичи)
#     "train_images_dir": "/kaggle/input/petfinder-pawpularity-score/train",
#     # папка с test-картинками (если используем image-фичи)
#     "test_images_dir": "/kaggle/input/petfinder-pawpularity-score/test",
#     # Если в таблице image_id уже содержит ".jpg", поставь IMAGE_EXT = ""
#     # расширение файлов картинок (если в таблице только id без расширения)
#     "image_ext":".jpg",
#     # колонка в таблице с именем файла картинки
#     "file_names_column": "Id",
#     # размер батча при обработке картинок
#     "batch_size": 32,
#     "clip_model_name": "openai/clip-vit-large-patch14",

    
#     # ---------- Text features ----------
#     # включить/выключить построение текстовых эмбеддингов
#     "text_enable": False,  # включить/выключить текстовые фичи

#     # любые текстовые колонки в основной train/test таблице
#     "text_columns": [],  # например ["title", "description"]

#     # внешние текстовые таблицы (id, text) в parquet/csv/tsv
#     # каждая запись:
#     # {
#     #   "name": "comments",        # логическое имя (для названия фич)
#     #   "train_path": "train_comments.parquet",
#     #   "test_path": "test_comments.parquet",
#     #   "format": "parquet",       # "parquet" / "csv" / "tsv" / "auto"
#     #   "id_column": "index",      # колонка id в этой таблице
#     #   "text_column": "text",     # колонка с текстом в этой таблице
#     #   "output_column": "comments_text"  # как назвать колонку в общей таблице (опц.)
#     # }
#     "text_external_tables": [{
#         "name": "reviews",
#         "train_path": "reviews.tsv",
#         "test_path": "reviews.tsv",
#         "format": "tsv",       # или "auto"
#         # имя колонки с уникальным идентификатором объекта
#     "id_column": "id",      # в этих файлах
#         "text_column": "text",     # колонка с текстом
#         "output_column": "comments_text"
#     }],

#     # тип эмбеддингов
#     # "bert"      — трансформер-эмбеддинги
#     # "tfidf_svd" — TF-IDF + TruncatedSVD
#     # тип текстовой модели: "bert" или "tfidf_svd"
#     "text_model_type": "tfidf_svd",

#     # варианты моделей (комментами, чтобы можно было быстро переключать)
#     # "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"  # RU + EN (универсал)
#     # "DeepPavlov/rubert-base-cased-sentence"                        # ru-only
#     # "sentence-transformers/all-mpnet-base-v2"                      # en-only
#     "text_bert_model_name": "DeepPavlov/rubert-base-cased-sentence",

#     # настройки BERT
#     "text_max_length": 256,
#     "text_batch_size": 32,
#     # настройки TF-IDF + SVD
#     "text_tfidf_max_features": 50000,
#     "text_svd_n_components": 256,

#     # ---------- CV ----------
#     "cv_n_splits": 5,
#     "cv_random_state": 42,
#     "cv_shuffle": True,
#     "cv_stratified": True,

#     # ---------- Models ----------
#     "use_catboost": True,
#     "use_lightgbm": False,
#     "use_xgboost": False,

#     "catboost_params": {
#         "iterations": 3500,
#         "learning_rate": 0.05,
#         "depth": 6,
#         "loss_function": "MultiClass", # "Logloss"
#         "eval_metric": "AUC",
#         "random_seed": 42,
#         "verbose": 100
#     },


#     "lgb_params": {
#         "objective": "multiclass", # "binary"
#         "eval_metric": "auc", #["auc", "binary_logloss"],
#         "learning_rate": 0.05,
#         "num_leaves": 31,
#         "feature_fraction": 0.9,
#         "bagging_fraction": 0.8,
#         "bagging_freq": 5,
#         "lambda_l2": 1.0,
#         "num_threads": 0,
#         "verbose": -1
#     },

#     "xgb_params": {
#         "objective": "multi:softmax", #"binary:logistic",
#         "eval_metric": "auc",
#         "learning_rate": 0.05,
#         "max_depth": 6,
#         "subsample": 0.8,
#         "colsample_bytree": 0.8,
#         "lambda": 1.0,
#         "alpha": 0.0,
#         "tree_method": "hist"
#     },

#     # ---------- Blending ----------
#     "blend_weights": {
#         "catboost": 1,
#         "lightgbm": 0.35,
#         "xgboost": 0.15
#     },

#     # ---------- Feature importance ----------
#     "compute_feature_importance": True,
#     "top_features_to_show": 50,

#     # ---------- Early stopping ----------
#     # параметр ранней остановки для бустингов
#     "early_stopping_rounds": 100,# Количество раундов без улучшения для ранней остановки
#     "device": "cuda",

#     "seed": 42,
#     "verbose": True
# }


In [ ]:
import os

DATA_FOLDER_PATH = '.'  # базовая директория проекта; все пути ниже будут строиться относительно неё

CONFIG = {
    # ---------- Paths ----------
    # Полный путь к train-файлу с табличными данными.
    # Обычно это CSV/TSV с признаками + target.
    "train_path": os.path.join(DATA_FOLDER_PATH, "/kaggle/input/petfinder-pawpularity-score/train.csv"),

    # Полный путь к test-файлу. Структура колонок должна в целом совпадать с train,
    # но без target (Pawpularity).
    "test_path": os.path.join(DATA_FOLDER_PATH, '/kaggle/input/petfinder-pawpularity-score/test.csv'),

    # Разделитель в текстовых файлах (здесь стандартный CSV).
    "sep": ",",

    # Имя колонки с уникальным идентификатором объекта.
    # Используется для:
    #  - мёрджа с внешними таблицами (тексты, гео и т.п.),
    #  - формирования сабмишна (Id, prediction).
    "id_column": "Id",

    # Имя колонки с целевой переменной (target).
    # Для Pawpularity — это "Pawpularity".
    "target_column": "Pawpularity",

    # Список колонок, которые нужно интерпретировать как datetime.
    # Если есть поля с датой/временем (created_at, date, timestamp) — добавляешь сюда.
    "datetime_columns": [],

    # Папка, куда будут сохраняться:
    #  - обученные модели,
    #  - логи,
    #  - важности признаков,
    #  - промежуточные артефакты.
    "output_dir": "tabular_boosting_output",

    # ---------- Task ----------
    # "binary" / "multiclass" / "regression"
    # Тип задачи:
    #  - "regression" — численный таргет (как здесь),
    #  - "binary" — бинарная классификация,
    #  - "multiclass" — мультикласс.
    "task_type": "regression",

    # ---------- Basic features ----------
    # Колонки, которые нужно сразу удалить из датафрейма после чтения,
    # до любой обработки. Например:
    #  - дубликаты id,
    #  - явно мусорные поля,
    #  - текст, который не планируется использовать.
    "basic_drop_columns": [],

    # Список колонок, которые нужно принудительно трактовать как категориальные,
    # даже если по типу/кол-ву уникальных они не выглядят как категории.
    # Например, бинарные флаги в 0/1, коды, id-шники категорий и т.п.
    "basic_as_categorical": [],

    # Порог "максимум уникальных значений" для того, чтобы числовую колонку
    # можно было считать категориальной (если auto-логика это использует).
    # Если уникальных значений > этого числа, колонка скорее всего будет считаться
    # числовой/текстовой, а не категориальной.
    "basic_max_cat_unique": 64,

    # Если True, для всех колонок из "datetime_columns" (и/или авто-определённых дат)
    # будут созданы дополнительные фичи: год, месяц, день, день недели и т.д.
    "basic_datetime_expand": False,

    # Какие именно компоненты даты генерировать, если basic_datetime_expand = True:
    #  - "year"  — год
    #  - "month" — месяц
    #  - "day"   — день месяца
    #  - "dow"   — день недели
    #  - "hour"  — час
    "basic_datetime_features": ["year", "month", "day", "dow", "hour"],

    # ---------- Categorical processing ----------
    # минимальный "размер" категории (по числу объектов и/или доле),
    # все категории меньше порога сливаются в 'other'

    # Минимальное число объектов в категории.
    # Категории с count < cat_min_count → объединяются в одну категорию 'other'.
    "cat_min_count": 20,          # минимальное кол-во объектов в категории

    # Минимальная доля объектов в категории (в долях от 0 до 1).
    # Если > 0, то категория с freq < cat_min_freq также считается редкой.
    # При 0.0 условие по частоте отключено (оставляем только порог по count).
    "cat_min_freq": 0.0,          # минимальная доля (0..1); 0 = игнорировать

    # если доля "редких" категорий в признаке >= этого порога — признак удаляется
    # То есть: после слияния редких категорий в 'other',
    # если доля 'other' >= cat_max_rare_share, колонка считается бесполезной
    # (слишком много мусорных значений) и выкидывается.
    "cat_max_rare_share": 0.98,

    # если после объединения в other уникальных всё ещё слишком много — признак удаляется
    # Например, если после группировки в 'other' всё равно > 500 уникальных категорий,
    # колонка воспринимается как слишком "широкая" и тоже дропается.
    "cat_max_unique_after_group": 500,

    # ---------- Post-feature service columns ----------
    # служебные колонки, которые нужно удалить после генерации фич
    # (исходные координаты, raw id, сырые адреса и т.п.)
    # Сюда можно добавить, например, 'category_geo_ref_lat', 'category_geo_ref_lon',
    # если после строительства geo-фич они больше не нужны в данных.
    "post_feature_drop_columns": [], # может быть удалить category_geo_ref_lat, category_geo_ref_lon

    # ---------- Address processing ----------
    # колонка-адрес, из которой нужно извлечь город
    # Если у тебя есть колонка-строка с полным адресом (улица, дом, город),
    # её имя задаётся тут. Если None — адресные фичи не считаются.
    "address_column": None,  

    # индекс элемента после split (например [-1] = последний)
    # можно указывать как отрицательный индекс
    # Пример: "ул. Пушкина, дом Колотушкина, Москва" split(",")
    # при address_city_index = -1 → "Москва".
    "address_city_index": -1,  

    # разделитель в строке адреса
    # По этому сепаратору адрес split-ится на части.
    "address_split_sep": ",",  

    # ---------- Aggregates ----------
    # Включить ли генерацию агрегатных фич:
    # группировки по колонкам и расчёт mean/std/min/max/... по числовым признакам.
    "agg_enable": False,

    # Список колонок, по которым делаем groupby.
    # Например: ["user_id"] или ["city", "category"].
    "agg_groupby_cols": [],

    # Список числовых колонок, для которых считаем агрегаты внутри групп.
    "agg_numeric_cols": [],

    # Какие агрегаты считать. Стандартные агрегирующие функции pandas:
    #  - "mean", "std", "min", "max", "sum", "median", "nunique", "count"
    "agg_aggs": ["mean", "std", "min", "max", "sum", "median", "nunique", "count"],

    # Префикс, который будет добавлен к именам новых агрегатных фич.
    # Например: "agg_user_id_amount_mean".
    "agg_prefix": "agg",

    # ---------- Geo features ----------
    # Включить/выключить гео-фичи. Если False — все поля ниже игнорируются.
    "geo_enable": False, 

    # Название колонки с широтой точки отправления (lat_from).
    "geo_lat_from_col": "pickup_lat",

    # Название колонки с долготой точки отправления (lon_from).
    "geo_lon_from_col": "pickup_lon",

    # Название колонки с широтой точки назначения (lat_to).
    "geo_lat_to_col": "dropoff_lat",

    # Название колонки с долготой точки назначения (lon_to).
    "geo_lon_to_col": "dropoff_lon",

    # Колонка с "референсной" широтой (например, центроид категории).
    # Обычно это создаётся через geo_reference (см. ниже).
    "geo_ref_lat": 'category_geo_ref_lat',

    # Колонка с "референсной" долготой.
    "geo_ref_lon": 'category_geo_ref_lon',

    # Префикс для всех сгенерированных гео-фич (дистанции, углы и т.п.).
    "geo_prefix": "geo",

    # дополнительные фичи (bearing, manhattan, dlat/dlon, midpoint)
    # Если True — помимо базового расстояния считаем:
    #  - bearing (азимут),
    #  - manhattan distance (по широте/долготе),
    #  - dlat/dlon (дельты координат),
    #  - midpoint (середина между точками).
    "geo_extra_enable": False,

    # Если координаты хранятся не в отдельных колонках, а в одной строке вида "lat,lon",
    # можно использовать geo_from_coord_col / geo_to_coord_col и geo_coord_string_sep.
    # Тогда lat/lon будут парситься из этих строк.
    "geo_from_coord_col": None,
    "geo_to_coord_col": None,
    "geo_coord_string_sep": ",",

    # ---------- Geo reference config ----------
    # Конфиг для вычисления "референсных" координат по категориям.
    "geo_reference": {
        # Включено ли построение референсных координат.
        "enabled": False,

        # Колонка с категорией (например, "city", "region", "shop_id"),
        # по которой будем усреднять координаты.
        "category_column": "category",

        # Название колонки со строкой координат "lat,lon" (если отдельные lat/lon не заданы).
        "coordinates_column": "coordinates",  # строка "lat,lon", если lat/lon не заданы

        # Колонка с широтой, если координаты уже разбиты по отдельным полям.
        # Если не None, берём широту отсюда, а не из coordinates_column.
        "lat_column": None,                  # <- если задать, берём отсюда широту

        # Колонка с долготой, аналогично lat_column.
        "lon_column": None,                  # <- если задать, берём отсюда долготу

        # Итоговое имя новой колонки с идентификатором/названием референсной точки.
        "output_column": "category_geo_ref",

        # Имя колонки с референсной широтой, которая потом может использоваться
        # как geo_ref_lat в основном конфиге.
        "output_lat_column": "category_geo_ref_lat",

        # Имя колонки с референсной долготой (аналогично широте).
        "output_lon_column": "category_geo_ref_lon",
    },

    # ---------- Correlation-based feature filtering ----------
    # включить/выключить фильтрацию по корреляции
    # Если True — будет строиться матрица корреляции между признаками,
    # и сильно скоррелированные фичи выше порога могут удаляться.
    "corr_enable": False,          

    # порог |Pearson|; 0.1, 0.05 и т.п.
    # Чем ниже порог — тем больше признаков будет считаться "сильно коррелирующими".
    # Здесь 0.95 — довольно высокий порог, то есть удаляться будут только почти дубликаты.
    "corr_pearson_min_abs": 0.95,   

    # считать ли phik
    # phik — более продвинутый аналог корреляции, хорошо работающий с категориальными фичами.
    "corr_use_phik": False,        

    # порог |phik| (аналогично Pearson, но для phik-метрики).
    "corr_phik_min_abs": 0.0,      

    #----------images---------
    # папка с train-картинками (если используем image-фичи)
    "train_images_dir": "/kaggle/input/petfinder-pawpularity-score/train",

    # папка с test-картинками (если используем image-фичи)
    "test_images_dir": "/kaggle/input/petfinder-pawpularity-score/test",

    # Если в таблице image_id уже содержит ".jpg", поставь IMAGE_EXT = ""
    # расширение файлов картинок (если в таблице только id без расширения)
    "image_ext": ".jpg",

    # колонка в таблице с именем файла картинки
    # Обычно совпадает с id, к которому дописывается image_ext.
    "file_names_column": "Id",

    # размер батча при обработке картинок
    # Чем больше батч, тем быстрее, но тем больше память видеокарты/CPU.
    "batch_size": 32,

    # имя CLIP-модели в HuggingFace, которая используется для извлечения image-эмбеддингов.
    "clip_model_name": "openai/clip-vit-large-patch14",

    
    # ---------- Text features ----------
    # включить/выключить построение текстовых эмбеддингов
    "text_enable": False,  # включить/выключить текстовые фичи

    # любые текстовые колонки в основной train/test таблице
    # Пример: ["title", "description"], если в train.csv есть такие поля.
    "text_columns": [],  # например ["title", "description"]

    # внешние текстовые таблицы (id, text) в parquet/csv/tsv
    # каждая запись:
    # {
    #   "name": "comments",        # логическое имя (для названия фич)
    #   "train_path": "train_comments.parquet",
    #   "test_path": "test_comments.parquet",
    #   "format": "parquet",       # "parquet" / "csv" / "tsv" / "auto"
    #   "id_column": "index",      # колонка id в этой таблице
    #   "text_column": "text",     # колонка с текстом в этой таблице
    #   "output_column": "comments_text"  # как назвать колонку в общей таблице (опц.)
    # }
    "text_external_tables": [{
        "name": "reviews",            # логическое имя внешней текстовой таблицы
        "train_path": "reviews.tsv",  # путь к текстам для train
        "test_path": "reviews.tsv",   # путь к текстам для test (может быть тот же файл)
        "format": "tsv",              # или "auto" — формат будет определён по расширению
        # имя колонки с уникальным идентификатором объекта
        "id_column": "id",            # в этих файлах
        # колонка с текстом
        "text_column": "text",        # колонка с самим текстом
        # имя новой колонки в общей табличке после merge по id_column
        "output_column": "comments_text"
    }],

    # тип эмбеддингов
    # "bert"      — трансформер-эмбеддинги
    # "tfidf_svd" — TF-IDF + TruncatedSVD
    # тип текстовой модели: "bert" или "tfidf_svd"
    "text_model_type": "tfidf_svd",

    # варианты моделей (комментами, чтобы можно было быстро переключать)
    # "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"  # RU + EN (универсал)
    # "DeepPavlov/rubert-base-cased-sentence"                        # ru-only
    # "sentence-transformers/all-mpnet-base-v2"                      # en-only
    "text_bert_model_name": "DeepPavlov/rubert-base-cased-sentence",

    # настройки BERT
    # Максимальная длина токенизированного текста (обрежет слишком длинные).
    "text_max_length": 256,
    # Размер батча при прогоне текстов через BERT.
    "text_batch_size": 32,

    # настройки TF-IDF + SVD
    # Максимальное число признаков TF-IDF (размер словаря).
    "text_tfidf_max_features": 50000,
    # Размерность итогового текстового эмбеддинга после SVD.
    "text_svd_n_components": 256,

    # ---------- CV ----------
    # Количество фолдов в кросс-валидации (KFold или StratifiedKFold).
    "cv_n_splits": 5,

    # random_state для генератора случайных чисел в KFold'е.
    "cv_random_state": 42,

    # Перемешивать ли данные перед разбиением на фолды.
    "cv_shuffle": True,

    # Использовать ли стратификацию по target (актуально для классификации).
    # Для регрессии обычно False, но можно оставить True, если реализована stratified-regression логика.
    "cv_stratified": True,

    # ---------- Models ----------
    # Флажки, какие модели участвуют в обучении/блендинге.
    "use_catboost": True,
    "use_lightgbm": False,
    "use_xgboost": False,

    # Параметры CatBoost.
    # ⚠️ Сейчас loss_function="MultiClass", eval_metric="AUC",
    # что больше подходит для мультикласса/классификации.
    # Для task_type="regression" обычно:
    #  - loss_function="RMSE" / "MAE"
    #  - eval_metric="RMSE" / "MAE" и т.п.
    "catboost_params": {
        "iterations": 3500,           # количество деревьев/итераций бустинга
        "learning_rate": 0.05,        # шаг обучения
        "depth": 6,                   # глубина деревьев
        "loss_function": "MultiClass", # "Logloss" / "RMSE" / "MAE" и др.
        "eval_metric": "AUC",         # метрика для отслеживания качества
        "random_seed": 42,            # сид для воспроизводимости
        "verbose": 100                # лог каждые 100 итераций
    },

    # Параметры LightGBM.
    # ⚠️ Здесь objective="multiclass", eval_metric="auc" — опять же конфиг под классификацию.
    # Для регрессии:
    #  - objective="regression"
    #  - eval_metric="rmse" / "mae" и т.п.
    "lgb_params": {
        "objective": "multiclass",    # "binary" / "regression" / "multiclass"
        "eval_metric": "auc",         # ["auc", "binary_logloss"] и т.п. для классификации
        "learning_rate": 0.05,        # шаг обучения
        "num_leaves": 31,             # сложность деревьев (кол-во листьев)
        "feature_fraction": 0.9,      # доля фич, используемых в каждом дереве
        "bagging_fraction": 0.8,      # доля объектов для bagging
        "bagging_freq": 5,            # как часто делать bagging
        "lambda_l2": 1.0,             # L2-регуляризация
        "num_threads": 0,             # число потоков (0 = авто)
        "verbose": -1                 # уровень логгирования
    },

    # Параметры XGBoost.
    # ⚠️ objective="multi:softmax", eval_metric="auc" — под мультикласс.
    # Для регрессии:
    #  - objective="reg:squarederror"
    #  - eval_metric="rmse" / "mae".
    "xgb_params": {
        "objective": "multi:softmax", # "binary:logistic", "reg:squarederror" и др.
        "eval_metric": "auc",
        "learning_rate": 0.05,        # шаг обучения
        "max_depth": 6,               # глубина деревьев
        "subsample": 0.8,             # доля объектов для каждого дерева
        "colsample_bytree": 0.8,      # доля признаков для каждого дерева
        "lambda": 1.0,                # L2-регуляризация
        "alpha": 0.0,                 # L1-регуляризация
        "tree_method": "hist"         # метод построения деревьев (hist — быстрый/памяте-эффективный)
    },

    # ---------- Blending ----------
    # Веса моделей в финальном ансамбле.
    # Если модель отключена (use_* = False), её вес можно игнорировать.
    # Итоговый предикт примерно: w_cat * y_cat + w_lgb * y_lgb + w_xgb * y_xgb.
    "blend_weights": {
        "catboost": 1,
        "lightgbm": 0.35,
        "xgboost": 0.15
    },

    # ---------- Feature importance ----------
    # Считать ли важности признаков после обучения бустингов.
    "compute_feature_importance": True,

    # Сколько топовых признаков показывать/логировать.
    "top_features_to_show": 50,

    # ---------- Early stopping ----------
    # параметр ранней остановки для бустингов
    # Количество итераций без улучшения метрики,
    # после которого обучение будет остановлено.
    "early_stopping_rounds": 100,  # Количество раундов без улучшения для ранней остановки

    # На каком устройстве считать:
    #  - "cuda" — GPU (если доступно),
    #  - "cpu" — процессор,
    #  - "mps" — Apple Silicon.
    "device": "cuda",

    # Общий сид для воспроизводимости (используется в разных местах).
    "seed": 42,

    # Если True — включено детальное логгирование (print/logging).
    "verbose": True
}


In [37]:
def setup_logging(output_dir: str) -> logging.Logger:
    os.makedirs(output_dir, exist_ok=True)
    log_path = os.path.join(output_dir, "training.log")

    logger = logging.getLogger("TABULAR_BOOSTING")
    logger.setLevel(logging.INFO)
    logger.handlers = []

    # Используем UTF-8 кодировку для файла, чтобы поддерживать эмодзи
    fh = logging.FileHandler(log_path, encoding='utf-8')
    fh.setLevel(logging.INFO)
    
    # Для консоли также используем UTF-8, если возможно
    import sys
    if sys.stdout.encoding != 'utf-8':
        # Если консоль не поддерживает UTF-8, создаем обертку
        class UTF8StreamHandler(logging.StreamHandler):
            def emit(self, record):
                try:
                    msg = self.format(record)
                    # Убираем эмодзи для консоли, если она не поддерживает UTF-8
                    import re
                    msg = re.sub(r'[^\x00-\x7F]+', '', msg)  # Удаляем не-ASCII символы
                    stream = self.stream
                    stream.write(msg + self.terminator)
                    self.flush()
                except Exception:
                    self.handleError(record)
        ch = UTF8StreamHandler()
    else:
        ch = logging.StreamHandler()
    ch.setLevel(logging.INFO)

    fmt = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
    fh.setFormatter(fmt)
    ch.setFormatter(fmt)

    logger.addHandler(fh)
    logger.addHandler(ch)

    logger.info("Logger initialized.")
    return logger


def log_config(logger: logging.Logger, config: Dict):
    logger.info("========== CONFIG ==========")
    logger.info(json.dumps(config, indent=2, ensure_ascii=False))

In [38]:
start_time = time.time()

output_dir = CONFIG["output_dir"]
os.makedirs(output_dir, exist_ok=True)
logger = setup_logging(output_dir)
log_config(logger, CONFIG)

2025-11-18 21:06:07,234 - INFO - Logger initialized.
INFO:TABULAR_BOOSTING:Logger initialized.
2025-11-18 21:06:07,236 - INFO - ========== CONFIG ==========
INFO:TABULAR_BOOSTING:========== CONFIG ==========
2025-11-18 21:06:07,238 - INFO - {
  "train_path": "/kaggle/input/petfinder-pawpularity-score/train.csv",
  "test_path": "/kaggle/input/petfinder-pawpularity-score/test.csv",
  "sep": ",",
  "id_column": "Id",
  "target_column": "Pawpularity",
  "datetime_columns": [],
  "output_dir": "tabular_boosting_output",
  "task_type": "regression",
  "basic_drop_columns": [],
  "basic_as_categorical": [],
  "basic_max_cat_unique": 64,
  "basic_datetime_expand": false,
  "basic_datetime_features": [
    "year",
    "month",
    "day",
    "dow",
    "hour"
  ],
  "cat_min_count": 20,
  "cat_min_freq": 0.0,
  "cat_max_rare_share": 0.98,
  "cat_max_unique_after_group": 500,
  "post_feature_drop_columns": [],
  "address_column": null,
  "address_city_index": -1,
  "address_split_sep": ",",
  "a

### Utils

In [39]:
def _ensure_lat_lon_from_single_column(
    df: pd.DataFrame,
    coord_col: str,
    lat_name: str,
    lon_name: str,
    sep: str = ","
) -> pd.DataFrame:
    """
    coord_col может быть:
      - строка вида "55.75,37.62"
      - список/кортеж [55.75, 37.62]
    создаём/перезаписываем колонки lat_name, lon_name
    """
    if coord_col not in df.columns:
        return df

    def _parse_one(x):
        if isinstance(x, (list, tuple)) and len(x) >= 2:
            return x[0], x[1]
        if isinstance(x, str):
            parts = x.split(sep)
            if len(parts) >= 2:
                try:
                    return float(parts[0]), float(parts[1])
                except Exception:
                    return np.nan, np.nan
        return np.nan, np.nan

    lat_list, lon_list = zip(*df[coord_col].map(_parse_one))
    df = df.copy()
    df[lat_name] = lat_list
    df[lon_name] = lon_list
    return df

# NEW: очистка имён фичей для бустингов (XGBoost особенно строгий)
def sanitize_feature_names(
    df: pd.DataFrame,
    logger: logging.Logger
) -> Tuple[pd.DataFrame, Dict]:
    """
    Преобразует имена колонок в строки без символов [, ], <, >,
    следит за уникальностью имён и возвращает:
      - новый DataFrame с переименованными колонками
      - mapping {старое_имя -> новое_имя}
    """
    old_cols = list(df.columns)
    new_cols: List[str] = []
    mapping: Dict = {}
    changed = False

    df = df.copy()

    for col in old_cols:
        new = str(col)

        # запрещённые символы (можно расширить при необходимости)
        new = new.replace("[", "(").replace("]", ")")
        new = new.replace("<", "_lt_").replace(">", "_gt_")

        # на всякий случай избавимся от очень "экзотики"
        # (например, пустые имена)
        if new.strip() == "":
            new = "feature"

        base = new
        k = 1
        # обеспечиваем уникальность
        while new in new_cols:
            new = f"{base}__{k}"
            k += 1

        if new != col:
            changed = True
        mapping[col] = new
        new_cols.append(new)

    df.columns = new_cols

    if changed:
        logger.info("Sanitized feature names for boosting models.")
        # при желании можно залогировать часть маппинга
        logger.info(
            "Example of feature name mapping: " +
            ", ".join(
                f"{k} -> {v}" for k, v in list(mapping.items())[:10]
            )
        )

    return df, mapping

In [40]:
class ImagesDataset:
    def __init__(self, file_paths, transform=None):
        self.file_paths = file_paths
        self.transform = transform
    
    def __getitem__(self, idx):
        ### read image

        file_path = self.file_paths[idx]
        image = Image.open(file_path).convert("RGB")
        image = self.transform(image)
        # DataLoader сам создаст batch dimension, поэтому unsqueeze не нужен
        return image
    
    def __len__(self):
        return len(self.file_paths)

In [41]:
def detect_categorical_columns(
    df: pd.DataFrame,
    max_unique: int,
    force_categorical: List[str]
) -> List[str]:
    # Проверяем, что max_unique не None
    if max_unique is None:
        max_unique = 64  # значение по умолчанию
    
    # Фильтруем None из force_categorical
    cats = set(c for c in force_categorical if c is not None and isinstance(c, str))
    
    for col in df.columns:
        # Пропускаем None или не-строковые имена колонок
        if col is None or not isinstance(col, str):
            continue
            
        try:
            if df[col].dtype == "object":
                cats.add(col)
            else:
                try:
                    nunique_val = df[col].nunique(dropna=True)
                    # Проверяем, что nunique_val не None и является числом
                    # Также проверяем, что max_unique - это число
                    if (nunique_val is not None and 
                        isinstance(nunique_val, (int, np.integer)) and
                        isinstance(max_unique, (int, np.integer, float))):
                        if int(nunique_val) <= int(max_unique):
                            cats.add(col)
                except (TypeError, ValueError) as e:
                    # Если не можем вычислить nunique, пропускаем колонку
                    continue
        except Exception:
            # Если возникают проблемы с колонкой, пропускаем её
            continue
    
    # Фильтруем None и не-строковые значения перед сортировкой
    # Также убеждаемся, что все значения - строки
    cats_filtered = []
    for c in cats:
        if c is not None:
            # Преобразуем в строку, если это не строка
            try:
                c_str = str(c) if not isinstance(c, str) else c
                if c_str and c_str.strip():  # Проверяем, что строка не пустая
                    cats_filtered.append(c_str)
            except Exception:
                continue
    
    # Сортируем только строки
    try:
        return sorted(cats_filtered)
    except TypeError as e:
        # Если все еще ошибка сортировки, возвращаем без сортировки
        return list(cats_filtered)


def process_categorical_features(
    X: pd.DataFrame,
    test: Optional[pd.DataFrame],
    cat_cols: List[str],
    config: Dict,
    logger: logging.Logger
) -> Tuple[pd.DataFrame, Optional[pd.DataFrame], List[str]]:
    """
    Для каждой категориальной фичи:
      1) категории с малой частотой/долей сливаются в 'other'
      2) если доля редких категорий слишком велика или
         даже после слияния слишком много уникальных — фича выкидывается.

    Возвращает:
      X_new, test_new, updated_cat_cols
    """
    if not cat_cols:
        return X, test, cat_cols

    min_count = int(config.get("cat_min_count", 0) or 0)
    min_freq = float(config.get("cat_min_freq", 0.0) or 0.0)
    max_rare_share = float(config.get("cat_max_rare_share", 1.0) or 1.0)
    max_unique_after = int(config.get("cat_max_unique_after_group", 10**9) or 10**9)

    if min_count <= 0 and min_freq <= 0.0 and max_rare_share >= 1.0:
        # фактически фильтрация отключена
        logger.info("Categorical processing: thresholds are trivial, skipping.")
        return X, test, cat_cols

    logger.info(
        "Categorical processing: "
        f"cat_min_count={min_count}, cat_min_freq={min_freq}, "
        f"cat_max_rare_share={max_rare_share}, "
        f"cat_max_unique_after_group={max_unique_after}"
    )

    X_new = X.copy()
    test_new = test.copy() if test is not None else None
    to_drop = []

    n = len(X_new)

    for col in tqdm(cat_cols, desc="Processing categorical features"):
        if col not in X_new.columns:
            continue

        vc = X_new[col].value_counts(dropna=False)
        total = vc.sum()

        # определяем "редкие" категории
        rare_mask = np.zeros(len(vc), dtype=bool)
        if min_count > 0:
            rare_mask |= (vc.values < min_count)
        if min_freq > 0.0:
            rare_mask |= (vc.values / total < min_freq)

        rare_cats = vc.index[rare_mask]
        rare_share = vc[rare_cats].sum() / total if len(rare_cats) > 0 else 0.0

        logger.info(
            f"[cat] {col}: unique={vc.size}, rare_cats={len(rare_cats)}, "
            f"rare_share={rare_share:.4f}"
        )

        # если редких слишком много — фича бесполезна, выкидываем
        if rare_share >= max_rare_share:
            logger.info(
                f"[cat] {col}: rare_share={rare_share:.4f} >= {max_rare_share}, "
                f"dropping whole feature."
            )
            to_drop.append(col)
            continue

        if len(rare_cats) == 0:
            # нечего объединять
            continue

        # объединяем редкие в 'other'
        other_label = "__OTHER__"
        X_new[col] = X_new[col].where(~X_new[col].isin(rare_cats), other_label)
        if test_new is not None:
            test_new[col] = test_new[col].where(~test_new[col].isin(rare_cats), other_label)

        # пересчитываем уникальные после объединения
        new_unique = X_new[col].nunique(dropna=True)
        if new_unique > max_unique_after:
            logger.info(
                f"[cat] {col}: unique_after={new_unique} > "
                f"cat_max_unique_after_group={max_unique_after}, dropping feature."
            )
            to_drop.append(col)

    if to_drop:
        logger.info(f"Dropping categorical features: {to_drop}")
        X_new = X_new.drop(columns=[c for c in to_drop if c in X_new.columns])
        if test_new is not None:
            test_new = test_new.drop(columns=[c for c in to_drop if c in test_new.columns])
        cat_cols = [c for c in cat_cols if c not in to_drop]

    return X_new, test_new, cat_cols


def expand_datetime_columns(
    df: pd.DataFrame,
    datetime_cols: List[str],
    features: List[str]
) -> pd.DataFrame:
    df = df.copy()
    for col in datetime_cols:
        if col not in df.columns:
            continue
        s = pd.to_datetime(df[col], errors="coerce")

        if "year" in features:
            df[f"{col}_year"] = s.dt.year.astype("Int64")
        if "month" in features:
            df[f"{col}_month"] = s.dt.month.astype("Int64")
        if "day" in features:
            df[f"{col}_day"] = s.dt.day.astype("Int64")
        if "dow" in features:
            df[f"{col}_dow"] = s.dt.dayofweek.astype("Int64")
        if "hour" in features:
            df[f"{col}_hour"] = s.dt.hour.astype("Int64")

    return df


def haversine_distance(
    lat1, lon1, lat2, lon2, radius: float = 6371.0
) -> np.ndarray:
    lat1 = np.radians(lat1.astype(float))
    lon1 = np.radians(lon1.astype(float))
    lat2 = np.radians(lat2.astype(float))
    lon2 = np.radians(lon2.astype(float))

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat / 2.0) ** 2 + \
        np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return radius * c


### Data processing

In [42]:
def load_data(config: Dict, logger: logging.Logger) -> Tuple[pd.DataFrame, Optional[pd.DataFrame]]:
    train_path = config["train_path"]
    test_path = config["test_path"]

    logger.info(f"📥 Loading train from {train_path}")
    train = pd.read_csv(train_path, sep=config["sep"])

    if test_path:
        logger.info(f"📥 Loading test from {test_path}")
        test = pd.read_csv(test_path, sep=config["sep"])
    else:
        test = None

    logger.info(f"Train shape: {train.shape}")
    if test is not None:
        logger.info(f"Test  shape: {test.shape}")

    return train, test

In [43]:
train_df, test_df = load_data(CONFIG, logger)

id_col = CONFIG["id_column"]
target_col = CONFIG["target_column"]

test_ids = None
if test_df is not None and id_col in test_df.columns:
    test_ids = test_df[id_col].copy()

y = train_df[target_col]
X = train_df.drop(columns=[target_col])

2025-11-18 21:06:21,703 - INFO -  Loading train from /kaggle/input/petfinder-pawpularity-score/train.csv
INFO:TABULAR_BOOSTING:📥 Loading train from /kaggle/input/petfinder-pawpularity-score/train.csv
2025-11-18 21:06:21,722 - INFO -  Loading test from /kaggle/input/petfinder-pawpularity-score/test.csv
INFO:TABULAR_BOOSTING:📥 Loading test from /kaggle/input/petfinder-pawpularity-score/test.csv
2025-11-18 21:06:21,725 - INFO - Train shape: (9912, 14)
INFO:TABULAR_BOOSTING:Train shape: (9912, 14)
2025-11-18 21:06:21,726 - INFO - Test  shape: (8, 13)
INFO:TABULAR_BOOSTING:Test  shape: (8, 13)


In [44]:
### Добавь сюда создание reference колонки для гео-фичей
# На основе усреднения координат при группировке по категориальной колонке
def fit_geo_reference(df: pd.DataFrame, config: Dict, logger: Optional[logging.Logger] = None) -> pd.DataFrame:
    """
    Строит DataFrame geo_ref_df, который сопоставляет каждой категории средние координаты.
    Здесь создаётся reference-колонка для гео-фичей:
      - <output_lat_column>, <output_lon_column> (float)
      - <output_column> как строка "lat,lon".
    """
    geo_cfg = config.get("geo_reference", {})
    cat_col = geo_cfg.get("category_column", "category")
    coord_col = geo_cfg.get("coordinates_column", "coordinates")
    out_col = geo_cfg.get("output_column", f"{cat_col}_geo_ref")
    out_lat_col = geo_cfg.get("output_lat_column", f"{cat_col}_geo_ref_lat")
    out_lon_col = geo_cfg.get("output_lon_column", f"{cat_col}_geo_ref_lon")

    lat_col_cfg = geo_cfg.get("lat_column")
    lon_col_cfg = geo_cfg.get("lon_column")

    df_work = df.copy()

    # 1) Берём lat/lon либо из явных колонок, либо парсим строку "lat,lon"
    if (
        lat_col_cfg
        and lon_col_cfg
        and lat_col_cfg in df_work.columns
        and lon_col_cfg in df_work.columns
    ):
        # Явные колонки широты/долготы
        df_work["_lat"] = pd.to_numeric(df_work[lat_col_cfg], errors="coerce")
        df_work["_lon"] = pd.to_numeric(df_work[lon_col_cfg], errors="coerce")
    else:
        # Парсим coordinates как строку "lat,lon" (можно с пробелами и знаками)
        coords_parsed = (
            df_work[coord_col]
            .astype(str)
            .str.extract(r"([+-]?\d+\.?\d*)\s*,\s*([+-]?\d+\.?\d*)")
        )
        df_work["_lat"] = pd.to_numeric(coords_parsed[0], errors="coerce")
        df_work["_lon"] = pd.to_numeric(coords_parsed[1], errors="coerce")

    # 2) Считаем средние координаты по группам
    geo_ref_df = (
        df_work.groupby(cat_col)[["_lat", "_lon"]]
        .mean()
        .rename(columns={"_lat": out_lat_col, "_lon": out_lon_col})
        .reset_index()
    )

    # 3) Строковая колонка "lat,lon" (для отладки/экспорта)
    geo_ref_df[out_col] = (
        geo_ref_df[out_lat_col].round(6).astype(str)
        + "," +
        geo_ref_df[out_lon_col].round(6).astype(str)
    )

    if logger:
        logger.info(
            f"🌍 Обучили reference-гео-центры по {cat_col}, всего групп: {geo_ref_df.shape[0]}"
        )
    return geo_ref_df

def add_geo_reference_column(
    df: pd.DataFrame,
    geo_ref_df: pd.DataFrame,
    config: Dict,
    logger: Optional[logging.Logger] = None
) -> pd.DataFrame:
    """
    Добавляет reference-гео-колонку(и) к df, используя geo_ref_df, посчитанную на train.
    """
    geo_cfg = config.get("geo_reference", {})
    cat_col = geo_cfg.get("category_column", "category")
    out_col = geo_cfg.get("output_column", f"{cat_col}_geo_ref")
    out_lat_col = geo_cfg.get("output_lat_column", f"{cat_col}_geo_ref_lat")
    out_lon_col = geo_cfg.get("output_lon_column", f"{cat_col}_geo_ref_lon")

    df = df.merge(geo_ref_df[[cat_col, out_col, out_lat_col, out_lon_col]], how="left", on=cat_col)
    n_missing = df[out_col].isna().sum()
    if logger:
        logger.info(
            f"✅ Добавлены reference-гео-колонки ({out_col}, {out_lat_col}, {out_lon_col}) "
            f"к df (geo reference по {cat_col}). Пропусков: {n_missing}"
        )
    return df


def maybe_apply_geo_reference(
    df: pd.DataFrame,
    config: Dict,
    logger: Optional[logging.Logger] = None,
    fit: bool = False,
    geo_ref_df: Optional[pd.DataFrame] = None
):
    """
    Обертка для train/test: если fit=True, вычисляет geo reference и добавляет в train,
    если fit=False (test), просто применяет geo_ref_df к df.
    Возвращает:
      - при fit=True: (df_with_ref, geo_ref_df)
      - при fit=False: df_with_ref
    """
    geo_cfg = config.get("geo_reference", {})
    if not geo_cfg.get("enabled", False):
        if logger:
            logger.info("🌎 Гео-reference выключен (config['geo_reference']['enabled']=False)")
        return (df, None) if fit else df

    if fit:
        geo_ref_df = fit_geo_reference(df, config, logger)
        df = add_geo_reference_column(df, geo_ref_df, config, logger)
        return df, geo_ref_df
    else:
        if geo_ref_df is None:
            raise ValueError("geo_ref_df must be passed for applying on test set")
        df = add_geo_reference_column(df, geo_ref_df, config, logger)
        return df

# --- Применяем reference-гео-фичи к train и test, если нужно ---
if CONFIG.get("geo_reference", {}).get("enabled", False):
    logger.info("🌎 Применяем гео-reference features...")
    train_df, geo_ref_df = maybe_apply_geo_reference(train_df, CONFIG, logger, fit=True)
    test_df = maybe_apply_geo_reference(test_df, CONFIG, logger, fit=False, geo_ref_df=geo_ref_df)

In [45]:
if CONFIG['task_type'] in ['multiclass', 'binary']:
    y = y.astype(int)

УДАЛЕНИЕ ID

In [46]:
# NOTE: ID column is intentionally NOT dropped here.
# Оно будет удалено позже, непосредственно перед обучением бустингов,
# чтобы сохранить ID для генерации фич (картинки/тексты/агрегации).

### Features generation

In [47]:
def generate_basic_features(
    df: pd.DataFrame,
    config: Dict,
    logger: Optional[logging.Logger] = None
) -> pd.DataFrame:
    df = df.copy()

    drop_cols = config["basic_drop_columns"]
    for c in drop_cols:
        if c in df.columns:
            if logger:
                logger.info(f"Dropping column: {c}")
            df = df.drop(columns=[c])

    if config["basic_datetime_expand"] and config["datetime_columns"]:
        if logger:
            logger.info(f"Expanding datetime columns: {config['datetime_columns']}")
        df = expand_datetime_columns(
            df,
            datetime_cols=config["datetime_columns"],
            features=config["basic_datetime_features"]
        )

    return df


def generate_aggregate_features(
    train: pd.DataFrame,
    test: Optional[pd.DataFrame],
    config: Dict,
    logger: Optional[logging.Logger] = None
) -> Tuple[pd.DataFrame, Optional[pd.DataFrame]]:
    if not config["agg_enable"]:
        if logger:
            logger.info("Aggregate features disabled in CONFIG.")
        return train, test

    groupby_cols = config["agg_groupby_cols"]
    if not groupby_cols:
        if logger:
            logger.info("No agg_groupby_cols specified — skipping aggregates.")
        return train, test

    if logger:
        logger.info(f"Generating aggregate features by {groupby_cols}")

    all_df = train if test is None else pd.concat([train, test], axis=0, ignore_index=True)

    if config["agg_numeric_cols"]:
        num_cols = [c for c in config["agg_numeric_cols"] if c in all_df.columns]
    else:
        num_cols = [
            c for c in all_df.columns
            if pd.api.types.is_numeric_dtype(all_df[c])
        ]
        for c in [config["target_column"], config["id_column"]]:
            if c in num_cols:
                num_cols.remove(c)

    aggs = config["agg_aggs"]
    prefix = config["agg_prefix"]

    if logger:
        logger.info(f"Aggregating numeric cols: {num_cols}")
        logger.info(f"Using aggs: {aggs}, prefix: {prefix}")

    grouped = all_df.groupby(groupby_cols)[num_cols].agg(aggs)
    grouped.columns = [
        f"{prefix}_" + "_".join(map(str, col)).strip()
        for col in grouped.columns.to_flat_index()
    ]
    grouped = grouped.reset_index()

    if logger:
        logger.info(f"Aggregate frame shape: {grouped.shape}")

    train_merged = train.merge(grouped, on=groupby_cols, how="left")
    test_merged = None
    if test is not None:
        test_merged = test.merge(grouped, on=groupby_cols, how="left")

    return train_merged, test_merged


def generate_geo_features(
    train: pd.DataFrame,
    test: Optional[pd.DataFrame],
    config: Dict,
    logger: Optional[logging.Logger] = None
) -> Tuple[pd.DataFrame, Optional[pd.DataFrame]]:
    """
    Базовые фичи:
      - geo_dist_from_to_km
      - geo_dist_from_ref_km, geo_dist_to_ref_km (если задан ref)
      - geo_lat_from_abs / geo_lon_from_abs / geo_lat_to_abs / geo_lon_to_abs

    Если geo_extra_enable=True:
      - geo_dlat / geo_dlon
      - geo_manhattan_km
      - geo_bearing_deg
      - geo_mid_lat / geo_mid_lon
    """
    if not config["geo_enable"]:
        if logger:
            logger.info("Geo features disabled in CONFIG.")
        return train, test

    lat_from_col = config["geo_lat_from_col"]
    lon_from_col = config["geo_lon_from_col"]
    lat_to_col = config["geo_lat_to_col"]
    lon_to_col = config["geo_lon_to_col"]
    prefix = config["geo_prefix"]

    from_coord_col = config.get("geo_from_coord_col")
    to_coord_col = config.get("geo_to_coord_col")
    coord_sep = config.get("geo_coord_string_sep", ",")

    if from_coord_col is not None:
        train = _ensure_lat_lon_from_single_column(
            train, from_coord_col, lat_from_col, lon_from_col, sep=coord_sep
        )
        if test is not None:
            test = _ensure_lat_lon_from_single_column(
                test, from_coord_col, lat_from_col, lon_from_col, sep=coord_sep
            )

    if to_coord_col is not None:
        train = _ensure_lat_lon_from_single_column(
            train, to_coord_col, lat_to_col, lon_to_col, sep=coord_sep
        )
        if test is not None:
            test = _ensure_lat_lon_from_single_column(
                test, to_coord_col, lat_to_col, lon_to_col, sep=coord_sep
            )


    for col in [lat_from_col, lon_from_col, lat_to_col, lon_to_col]:
        if col and col not in train.columns:
            raise ValueError(f"Column {col} not found in train for geo features")

    if logger:
        logger.info(f"Generating geo features with backend: {GEO_BACKEND}")

    def _distance_rowwise(
        lat1: pd.Series, lon1: pd.Series, lat2: pd.Series, lon2: pd.Series
    ) -> np.ndarray:
        if GEO_BACKEND == "geopy":
            def _one(a, b, c, d):
                if pd.isna(a) or pd.isna(b) or pd.isna(c) or pd.isna(d):
                    return np.nan
                return geodesic((a, b), (c, d)).km
            return np.vectorize(_one)(lat1, lon1, lat2, lon2)
        else:
            return haversine_distance(lat1, lon1, lat2, lon2)

    def _bearing(
        lat1: pd.Series, lon1: pd.Series, lat2: pd.Series, lon2: pd.Series
    ) -> np.ndarray:
        # initial bearing (degrees)
        lat1_r = np.radians(lat1.astype(float))
        lat2_r = np.radians(lat2.astype(float))
        dlon_r = np.radians(lon2.astype(float) - lon1.astype(float))

        x = np.sin(dlon_r) * np.cos(lat2_r)
        y = np.cos(lat1_r) * np.sin(lat2_r) - np.sin(lat1_r) * np.cos(lat2_r) * np.cos(dlon_r)
        brng = np.degrees(np.arctan2(x, y))
        brng = (brng + 360) % 360
        return brng

    def _apply_geo(df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()

        df[f"{prefix}_dist_from_to_km"] = _distance_rowwise(
            df[lat_from_col], df[lon_from_col],
            df[lat_to_col], df[lon_to_col]
        )

        ref_lat = config["geo_ref_lat"]
        ref_lon = config["geo_ref_lon"]
        if ref_lat is not None and ref_lon is not None:
            df[f"{prefix}_dist_from_ref_km"] = _distance_rowwise(
                df[lat_from_col], df[lon_from_col],
                pd.Series(ref_lat, index=df.index),
                pd.Series(ref_lon, index=df.index),
            )
            df[f"{prefix}_dist_to_ref_km"] = _distance_rowwise(
                df[lat_to_col], df[lon_to_col],
                pd.Series(ref_lat, index=df.index),
                pd.Series(ref_lon, index=df.index),
            )

        df[f"{prefix}_lat_from_abs"] = df[lat_from_col].abs()
        df[f"{prefix}_lon_from_abs"] = df[lon_from_col].abs()
        df[f"{prefix}_lat_to_abs"] = df[lat_to_col].abs()
        df[f"{prefix}_lon_to_abs"] = df[lon_to_col].abs()

        if config.get("geo_extra_enable", False):
            df[f"{prefix}_dlat"] = df[lat_to_col] - df[lat_from_col]
            df[f"{prefix}_dlon"] = df[lon_to_col] - df[lon_from_col]

            df[f"{prefix}_manhattan_km"] = (
                haversine_distance(df[lat_from_col], df[lon_from_col], df[lat_to_col], df[lon_from_col]) +
                haversine_distance(df[lat_to_col], df[lon_from_col], df[lat_to_col], df[lon_to_col])
            )

            df[f"{prefix}_bearing_deg"] = _bearing(
                df[lat_from_col], df[lon_from_col],
                df[lat_to_col], df[lon_to_col]
            )

            df[f"{prefix}_mid_lat"] = (df[lat_from_col] + df[lat_to_col]) / 2.0
            df[f"{prefix}_mid_lon"] = (df[lon_from_col] + df[lon_to_col]) / 2.0

        return df

    train_geo = _apply_geo(train)
    test_geo = _apply_geo(test) if test is not None else None
    return train_geo, test_geo


def process_address_extract_city(
    df: pd.DataFrame,
    column: str,
    city_index: int,
    sep: str,
    logger: logging.Logger
) -> pd.DataFrame:
    """
    Здесь извлекается колонка 'город':
      - address -> список частей (split)
      - из списка берём city_index (по умолчанию -1 = последний элемент)
      - создаём новую колонку address_city
      - она впоследствии становится категориальной и проходит
        такую же обработку, как и остальные категориальные фичи
    """
    if column not in df.columns:
        logger.warning(f"Address column '{column}' not found, skipping city extraction.")
        return df

    df = df.copy()

    def _extract(addr):
        if not isinstance(addr, str):
            return None
        parts = [p.strip() for p in addr.split(sep)]
        if len(parts) == 0:
            return None
        try:
            return parts[city_index]
        except Exception:
            # если индекс вне диапазона
            return parts[-1]

    df["address_city"] = df[column].map(_extract)
    return df


In [48]:
import os

def _normalize_image_name(name: str, default_ext: str | None = None) -> str:
    """
    Приводим имя файла к нормальному виду:
    - убираем пробелы/переводы строк
    - если нет расширения, добавляем default_ext или CONFIG["image_ext"]
    """
    name = str(name).strip()
    base = os.path.basename(name)
    root, ext = os.path.splitext(base)
    if ext == "":
        if default_ext is None:
            default_ext = CONFIG.get("image_ext", "")
        return root + (default_ext or "")
    return root + ext


def images_features(X, test_df):
    """
    Добавляет к табличным фичам эмбеддинги картинок, посчитанные CLIP-моделью от OpenAI
    (через HuggingFace Transformers, только image-encoder).

    Ожидается, что в CONFIG заданы:
      - 'file_names_column'   — колонка с именами файлов
      - 'train_images_dir'    — папка с train-картинками
      - 'test_images_dir'     — папка с test-картинками
      - 'batch_size'          — размер батча для инференса
      - 'clip_model_name'     — имя CLIP-модели в HuggingFace (по умолчанию ViT-L/14)
    """
    file_name_column = CONFIG['file_names_column']

    if file_name_column not in X.columns:
        raise KeyError(f"CONFIG['file_names_column']={file_name_column!r} нет в train")

    if test_df is not None and file_name_column not in test_df.columns:
        raise KeyError(f"CONFIG['file_names_column']={file_name_column!r} нет в test")

    # ----------------- CLIP BACKBONE (image encoder only) -----------------
    if hasattr(torch.backends, "mps") and torch.backends.mps.is_available():
        device = "mps"
    else:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    clip_model_name = CONFIG.get("clip_model_name", "openai/clip-vit-large-patch14")
    # Загружаем модель и процессор CLIP
    model = CLIPModel.from_pretrained(clip_model_name)
    processor = CLIPProcessor.from_pretrained(clip_model_name)

    model = model.to(device)
    model.eval()

    # ----------------- TRAIN IMAGES -----------------
    train_file_paths = [
        os.path.join(
            CONFIG['train_images_dir'],
            _normalize_image_name(file_name, CONFIG.get("image_ext", ""))
        )
        for file_name in X[file_name_column]
    ]
    train_images_part = process_images(train_file_paths, model, processor, device, name="train_clip")
    train_images_part.index = X.index
    X = pd.concat([X, train_images_part], axis=1)

    # ----------------- TEST IMAGES ------------------
    if test_df is not None:
        test_file_paths = [
            os.path.join(
                CONFIG['test_images_dir'],
                _normalize_image_name(file_name, CONFIG.get("image_ext", ""))
            )
            for file_name in test_df[file_name_column]
        ]
        test_images_part = process_images(test_file_paths, model, processor, device, name="test_clip")
        test_images_part.index = test_df.index
        test_df = pd.concat([test_df, test_images_part], axis=1)

    # (опционально) выбрасываем колонку с именем файла, чтобы она не ушла в бустинги
    for df in (X, test_df):
        if df is not None and file_name_column in df.columns:
            df.drop(columns=[file_name_column], inplace=True)

    return X, test_df


def process_images(file_paths, model, processor, device, name):
    """
    Считает эмбеддинги для списка путей к картинкам одной CLIP-моделью.
    Использует CLIPProcessor + CLIPModel.get_image_features.
    """
    batch_size = CONFIG.get("batch_size", 32)

    collect_embs = []
    if not file_paths:
        return pd.DataFrame()

    for start in tqdm(
        range(0, len(file_paths), batch_size),
        total=(len(file_paths) + batch_size - 1) // batch_size,
        desc=f"Images {name} process..."
    ):
        batch_paths = file_paths[start:start + batch_size]
        images = []
        for path in batch_paths:
            try:
                img = Image.open(path).convert("RGB")
            except Exception:
                # если не удалось открыть картинку — используем заглушку
                img = Image.new("RGB", (224, 224), color=0)
            images.append(img)

        with torch.no_grad():
            inputs = processor(images=images, return_tensors="pt", padding=True).to(device)
            embs = model.get_image_features(**inputs)

            # L2-нормализация по строкам (num_samples x dim)
            embs = embs / (embs.norm(dim=-1, keepdim=True) + 1e-12)

        collect_embs.extend(embs.cpu().numpy().tolist())

    if not collect_embs:
        # На всякий случай, чтобы не упасть на пустом списке
        return pd.DataFrame()

    n_feat = len(collect_embs[0])
    return pd.DataFrame(collect_embs, columns=[f"emb_{i}" for i in range(n_feat)])


ТЕКСТОВЫЕ ФИЧИ

In [49]:
def get_text_device():
    global TEXT_DEVICE
    if TEXT_DEVICE is None:
        TEXT_DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    return TEXT_DEVICE


def init_bert_model():
    """
    Инициализируем BERT-модель и токенайзер один раз.
    """
    global TEXT_MODEL, TEXT_TOKENIZER, TEXT_DEVICE

    if TEXT_MODEL is not None:
        return

    model_name = CONFIG.get(
        "text_bert_model_name",
        "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
    )
    TEXT_DEVICE = get_text_device()

    TEXT_TOKENIZER = AutoTokenizer.from_pretrained(model_name)
    TEXT_MODEL = AutoModel.from_pretrained(model_name)
    TEXT_MODEL.to(TEXT_DEVICE)
    TEXT_MODEL.eval()
    
def _detect_format_from_path(path: str) -> str:
    ext = os.path.splitext(path)[1].lower()
    if ext == ".parquet":
        return "parquet"
    if ext == ".csv":
        return "csv"
    if ext == ".tsv":
        return "tsv"
    # по умолчанию считаем csv
    return "csv"


def load_text_table(path: str, fmt: str | None, id_column: str, text_column: str) -> pd.DataFrame:
    """
    Загружаем внешнюю таблицу с текстом.
    Поддерживаем parquet/csv/tsv. format="auto" -> по расширению.
    Возвращаем только столбцы [id_column, text_column].
    """
    if fmt is None or fmt == "auto":
        fmt = _detect_format_from_path(path)

    if fmt == "parquet":
        df = pd.read_parquet(path)
    elif fmt == "tsv":
        df = pd.read_csv(path, sep="\t")
    else:  # "csv" и всё остальное по-умолчанию
        df = pd.read_csv(path)

    # Явно проверяем, что нужные колонки есть
    missing = [c for c in (id_column, text_column) if c not in df.columns]
    if missing:
        raise ValueError(
            f"Columns {missing} not found in external text table {path}. "
            f"Available columns: {list(df.columns)}"
        )

    return df[[id_column, text_column]].copy()

def attach_external_text_tables(X: pd.DataFrame, test_df: pd.DataFrame):
    """Подцепляем внешние текстовые таблицы (CONFIG['text_external_tables']) к X и test_df.

    Важно: если во внешней таблице несколько строк на один и тот же id,
    мы сначала агрегируем их в ОДНУ строку (конкатенируем тексты через пробел),
    чтобы merge НЕ раздувал количество строк в основной таблице.
    """
    external_cfgs = CONFIG.get("text_external_tables", []) or []
    if not external_cfgs:
        return X, test_df, []

    main_id_col = CONFIG.get("id_column", "index")
    added_cols: list[str] = []

    for cfg in external_cfgs:
        name = cfg.get("name", "ext")

        train_path = cfg["train_path"]
        # если test_path не указан — используем тот же файл
        test_path = cfg.get("test_path", train_path)

        fmt = cfg.get("format", "auto")

        # ID-колонка в external-таблице: по умолчанию такая же, как в основной
        ext_id_col = cfg.get("id_column", main_id_col)

        text_col_ext = cfg["text_column"]
        output_col = cfg.get("output_column", f"text_ext_{name}")

        # --- TRAIN ---
        ext_train = load_text_table(train_path, fmt, ext_id_col, text_col_ext)
        ext_train = ext_train.rename(columns={text_col_ext: output_col})

        # NEW: агрегируем несколько текстов по одному id в одну строку
        if ext_train[ext_id_col].duplicated().any():
            ext_train = (
                ext_train
                .groupby(ext_id_col, as_index=False)[output_col]
                .agg(lambda s: " ".join(map(str, s)))
            )

        # Проверяем, есть ли main_id_col в колонках X
        # Если нет, используем индекс для merge
        if main_id_col in X.columns:
            # ID колонка есть в X
            if main_id_col == ext_id_col:
                # Имя ID одинаковое в основной и внешней таблице
                X = X.merge(
                    ext_train[[ext_id_col, output_col]],
                    on=ext_id_col,
                    how="left"
                )
            else:
                # Имя ID в основной и внешней таблицах разное
                X = X.merge(
                    ext_train[[ext_id_col, output_col]],
                    left_on=main_id_col,
                    right_on=ext_id_col,
                    how="left"
                ).drop(columns=[ext_id_col])
        else:
            # ID колонки нет в X, используем индекс
            # Устанавливаем ext_id_col как индекс во внешней таблице
            ext_train_indexed = ext_train.set_index(ext_id_col)[[output_col]]
            X = X.merge(
                ext_train_indexed,
                left_index=True,
                right_index=True,
                how="left"
            )

        # --- TEST ---
        if test_df is not None:
            ext_test = load_text_table(test_path, fmt, ext_id_col, text_col_ext)
            ext_test = ext_test.rename(columns={text_col_ext: output_col})

            # NEW: агрегируем несколько текстов по одному id в одну строку
            if ext_test[ext_id_col].duplicated().any():
                ext_test = (
                    ext_test
                    .groupby(ext_id_col, as_index=False)[output_col]
                    .agg(lambda s: " ".join(map(str, s)))
                )

            # Проверяем, есть ли main_id_col в колонках test_df
            if main_id_col in test_df.columns:
                if main_id_col == ext_id_col:
                    test_df = test_df.merge(
                        ext_test[[ext_id_col, output_col]],
                        on=ext_id_col,
                        how="left"
                    )
                else:
                    test_df = test_df.merge(
                        ext_test[[ext_id_col, output_col]],
                        left_on=main_id_col,
                        right_on=ext_id_col,
                        how="left"
                    ).drop(columns=[ext_id_col])
            else:
                # ID колонки нет в test_df, используем индекс
                # Устанавливаем ext_id_col как индекс во внешней таблице
                ext_test_indexed = ext_test.set_index(ext_id_col)[[output_col]]
                test_df = test_df.merge(
                    ext_test_indexed,
                    left_index=True,
                    right_index=True,
                    how="left"
                )

        added_cols.append(output_col)

    return X, test_df, added_cols


def build_bert_embeddings_for_field(field_name: str,
                                    train_texts: list[str],
                                    test_texts: list[str]) -> tuple[np.ndarray, np.ndarray]:
    """
    Строим эмбеддинги BERT для одного логического текстового поля.
    BERT-модель общая для всех полей, но считаем отдельно для каждого.
    """
    init_bert_model()

    device = get_text_device()
    max_len = CONFIG.get("text_max_length", 256)
    batch_size = CONFIG.get("text_batch_size", 32)

    def encode_list(texts: list[str], desc: str) -> np.ndarray:
        all_embs = []
        for i in tqdm(range(0, len(texts), batch_size), desc=f"{desc} [{field_name}]"):
            batch = texts[i:i + batch_size]

            enc = TEXT_TOKENIZER(
                batch,
                padding=True,
                truncation=True,
                max_length=max_len,
                return_tensors="pt"
            )
            enc = {k: v.to(device) for k, v in enc.items()}

            with torch.no_grad():
                outputs = TEXT_MODEL(**enc)
                last_hidden = outputs.last_hidden_state  # [bs, seq_len, hidden]

                mask = enc["attention_mask"].unsqueeze(-1).expand(last_hidden.size())
                masked = last_hidden * mask

                summed = masked.sum(dim=1)
                counts = mask.sum(dim=1).clamp(min=1)
                mean_pooled = summed / counts  # [bs, hidden]

                embs = mean_pooled.cpu().numpy()

                # L2 нормализация
                norms = np.linalg.norm(embs, axis=1, keepdims=True)
                norms = np.clip(norms, 1e-12, None)
                embs = embs / norms

                all_embs.append(embs)

        return np.vstack(all_embs) if len(all_embs) > 0 else np.zeros((0, TEXT_MODEL.config.hidden_size))

    train_embs = encode_list(train_texts, "BERT text embeddings (train)")
    test_embs = encode_list(test_texts, "BERT text embeddings (test)")

    return train_embs, test_embs
def build_tfidf_svd_embeddings_for_field(field_name: str,
                                         train_texts: list[str],
                                         test_texts: list[str],
                                         logger: logging.Logger) -> tuple[np.ndarray, np.ndarray]:
    """
    TF-IDF (uni+bi-grams) -> TruncatedSVD до компактного вектора -> L2-нормализация.
    Для каждого текстового поля свой TF-IDF и SVD, чтобы не мешать разные источники.
    """
    max_features = CONFIG.get("text_tfidf_max_features", 50000)
    n_components = CONFIG.get("text_svd_n_components", 256)
    random_state = CONFIG.get("seed", 42)

    vect = TFIDF_VECTORIZERS.get(field_name)
    svd = TFIDF_SVDS.get(field_name)

    if vect is None or svd is None:
        vect = TfidfVectorizer(
            max_features=max_features,
            ngram_range=(1, 2)
        )
        logger.info(f"TF-IDF fitting for field '{field_name}'...")
        train_tfidf = vect.fit_transform(train_texts)

        svd = TruncatedSVD(
            n_components=n_components,
            random_state=random_state
        )
        logger.info(f"SVD fitting for field '{field_name}'...")
        train_embs = svd.fit_transform(train_tfidf)

        TFIDF_VECTORIZERS[field_name] = vect
        TFIDF_SVDS[field_name] = svd
    else:
        train_tfidf = vect.transform(train_texts)
        train_embs = svd.transform(train_tfidf)

    test_tfidf = vect.transform(test_texts)
    test_embs = svd.transform(test_tfidf)

    # L2-нормализация
    def l2_norm(x: np.ndarray) -> np.ndarray:
        norms = np.linalg.norm(x, axis=1, keepdims=True)
        norms = np.where(norms == 0, 1, norms)
        return x / norms

    train_embs = l2_norm(train_embs)
    test_embs = l2_norm(test_embs)

    return train_embs, test_embs


In [50]:
def text_features(X: pd.DataFrame, test_df: pd.DataFrame, logger: logging.Logger):
    """
    1) Приклеиваем внешние текстовые таблицы (parquet/csv/tsv) по id.
    2) Для каждой текстовой колонки строим отдельный эмбеддинг (BERT или TF-IDF+SVD).
    3) Конкатим всё к X и test_df.
    """
    if not CONFIG.get("text_enable", False):
        return X, test_df

    logger.info("Text feature extraction starts...")

    # 1. Подцепляем внешние текстовые таблицы
    X, test_df, external_cols = attach_external_text_tables(X, test_df)

    # 2. Собираем финальный список текстовых колонок
    base_text_cols = CONFIG.get("text_columns", []) or []
    # фильтруем только те, которые реально есть в таблице
    base_text_cols = [c for c in base_text_cols if c in X.columns]

    all_text_cols = base_text_cols + external_cols

    if not all_text_cols:
        logger.warning("No text columns found for text_features. Skipping.")
        return X, test_df

    model_type = CONFIG.get("text_model_type", "bert")

    # 3. Для каждой текстовой колонки строим свои эмбеддинги и конкатим
    for col in tqdm(all_text_cols, desc="Building text embeddings"):
        logger.info(f"Building text embeddings for column: {col}")

        # собираем тексты (строка -> str) - ВАЖНО: сохраняем порядок через индексы
        train_texts = X[col].fillna("").astype(str).tolist()
        test_texts = test_df[col].fillna("").astype(str).tolist()

        # Проверка размеров перед созданием эмбеддингов
        if len(train_texts) != len(X):
            raise ValueError(f"Train texts length {len(train_texts)} != X length {len(X)} for column {col}")
        if len(test_texts) != len(test_df):
            raise ValueError(f"Test texts length {len(test_texts)} != test_df length {len(test_df)} for column {col}")

        # проверка, а не все ли пустые
        if all(t == "" for t in train_texts) and all(t == "" for t in test_texts):
            logger.warning(f"Column '{col}' has only empty texts. Skipping.")
            continue

        field_name = col  # можно потом маппить/переименовать если хочешь

        if model_type == "bert":
            train_embs, test_embs = build_bert_embeddings_for_field(field_name, train_texts, test_texts)
        elif model_type == "tfidf_svd":
            train_embs, test_embs = build_tfidf_svd_embeddings_for_field(field_name, train_texts, test_texts, logger)
        else:
            raise ValueError(f"Unknown text_model_type: {model_type}")

        # Проверка размеров эмбеддингов
        if train_embs.shape[0] != len(X):
            raise ValueError(f"Train embeddings shape {train_embs.shape[0]} != X length {len(X)} for column {col}")
        if test_embs.shape[0] != len(test_df):
            raise ValueError(f"Test embeddings shape {test_embs.shape[0]} != test_df length {len(test_df)} for column {col}")

        dim = train_embs.shape[1]
        # префикс с названием колонки, чтобы понимать источник
        prefix = f"text_{field_name}_emb"
        cols = [f"{prefix}_{i}" for i in range(dim)]

        # Создаем DataFrame с правильными индексами
        train_text_df = pd.DataFrame(train_embs, columns=cols, index=X.index)
        test_text_df = pd.DataFrame(test_embs, columns=cols, index=test_df.index)

        X = pd.concat([X, train_text_df], axis=1)
        test_df = pd.concat([test_df, test_text_df], axis=1)

        logger.info(f"Text features for '{col}' added: {dim} dims ({prefix}_*)")

    return X, test_df


In [51]:
def prepare_all_features(X, test_df, config, logger):
    # КРИТИЧЕСКАЯ ПРОВЕРКА: сохраняем исходные размеры
    original_X_size = len(X)
    original_test_size = len(test_df) if test_df is not None else 0
    logger.info(f"🔍 ORIGINAL SIZES: X={original_X_size}, test_df={original_test_size}")
    
    # ----- Features -----  
    logger.info("🔧 Generating basic features...")
    logger.info(f"X shape before basic: {X.shape}")
    if test_df is not None:
        logger.info(f"test_df shape before basic: {test_df.shape}")
    
    X = generate_basic_features(X, config, logger)
    if test_df is not None:
        test_df = generate_basic_features(test_df, config, logger)
    
    logger.info(f"X shape after basic: {X.shape}")
    if test_df is not None:
        logger.info(f"test_df shape after basic: {test_df.shape}")
        if len(test_df) != original_test_size:
            raise ValueError(f"❌ test_df size changed in generate_basic_features: {original_test_size} -> {len(test_df)}")

    logger.info("📊 Generating aggregate features...")
    logger.info(f"X shape before aggregate: {X.shape}")
    if test_df is not None:
        logger.info(f"test_df shape before aggregate: {test_df.shape}")
    
    X, test_df = generate_aggregate_features(X, test_df, config, logger)
    
    logger.info(f"X shape after aggregate: {X.shape}")
    if test_df is not None:
        logger.info(f"test_df shape after aggregate: {test_df.shape}")
        if len(test_df) != original_test_size:
            raise ValueError(f"❌ test_df size changed in generate_aggregate_features: {original_test_size} -> {len(test_df)}")

    logger.info("🗺  Generating geo features...")
    logger.info(f"X shape before geo: {X.shape}")
    if test_df is not None:
        logger.info(f"test_df shape before geo: {test_df.shape}")
    
    X, test_df = generate_geo_features(X, test_df, config, logger)
    
    logger.info(f"X shape after geo: {X.shape}")
    if test_df is not None:
        logger.info(f"test_df shape after geo: {test_df.shape}")
        if len(test_df) != original_test_size:
            raise ValueError(f"❌ test_df size changed in generate_geo_features: {original_test_size} -> {len(test_df)}")
    
    if config['train_images_dir']:
        logger.info("Images process starts...")
        X, test_df = images_features(X, test_df)
    
    if CONFIG.get("text_enable", False):
        logger.info("Text process starts...")
        logger.info(f"X shape before text: {X.shape}")
        if test_df is not None:
            logger.info(f"test_df shape before text: {test_df.shape}")
        
        X, test_df = text_features(X, test_df, logger)
        
        logger.info(f"X shape after text: {X.shape}")
        if test_df is not None:
            logger.info(f"test_df shape after text: {test_df.shape}")
            if len(test_df) != original_test_size:
                raise ValueError(f"❌ test_df size changed in text_features: {original_test_size} -> {len(test_df)}")

    # ----- Address → city extraction -----
    addr_col = config.get("address_column")
    if addr_col:
        logger.info(f"🏙  Extracting city from address column '{addr_col}' ...")
        logger.info(f"X shape before address: {X.shape}")
        if test_df is not None:
            logger.info(f"test_df shape before address: {test_df.shape}")
        
        city_index = config.get("address_city_index", -1)
        sep = config.get("address_split_sep", ",")
        X = process_address_extract_city(X, addr_col, city_index, sep, logger)
        if test_df is not None:
            test_df = process_address_extract_city(test_df, addr_col, city_index, sep, logger)
        
        logger.info(f"X shape after address: {X.shape}")
        if test_df is not None:
            logger.info(f"test_df shape after address: {test_df.shape}")
            if len(test_df) != original_test_size:
                raise ValueError(f"❌ test_df size changed in process_address_extract_city: {original_test_size} -> {len(test_df)}")

    # ----- Categorical detection -----
    logger.info("🔎 Detecting categorical columns...")
    logger.info(f"X shape before categorical detection: {X.shape}")
    if test_df is not None:
        logger.info(f"test_df shape before categorical detection: {test_df.shape}")
    
    concat_df = pd.concat([X] if test_df is None else [X, test_df], ignore_index=False)
    cat_cols = detect_categorical_columns(
        concat_df,
        max_unique=config["basic_max_cat_unique"],
        force_categorical=config["basic_as_categorical"]
    )
    cat_cols = [c for c in cat_cols if c in X.columns]
    logger.info(f"Categorical columns: {cat_cols}")

    # ----- Categorical post-processing: rare → 'other' -----
    logger.info("🧩 Processing categorical features (merge rare into 'other')...")
    logger.info(f"X shape before categorical processing: {X.shape}")
    if test_df is not None:
        logger.info(f"test_df shape before categorical processing: {test_df.shape}")
    
    X, test_df, cat_cols = process_categorical_features(X, test_df, cat_cols, config, logger)
    logger.info(f"Categorical columns after processing: {cat_cols}")
    
    logger.info(f"X shape after categorical processing: {X.shape}")
    if test_df is not None:
        logger.info(f"test_df shape after categorical processing: {test_df.shape}")
        if len(test_df) != original_test_size:
            raise ValueError(f"❌ test_df size changed in process_categorical_features: {original_test_size} -> {len(test_df)}")

    # ----- Post-feature service columns drop -----
    post_drop = config.get("post_feature_drop_columns", [])
    if post_drop:
        logger.info(f"🧹 Dropping post-feature service columns: {post_drop}")
        X = X.drop(columns=[c for c in post_drop if c in X.columns])
        if test_df is not None:
            test_df = test_df.drop(columns=[c for c in post_drop if c in test_df.columns])
        # обновляем список категориальных
        cat_cols = [c for c in cat_cols if c not in post_drop]

    # Обработка NaN: сначала категориальные, потом числовые
    # Для категориальных используем специальное значение
    for col_name in cat_cols:
        if col_name in X.columns:
            X[col_name] = X[col_name].fillna("__MISSING__").astype(str)
        if test_df is not None and col_name in test_df.columns:
            test_df[col_name] = test_df[col_name].fillna("__MISSING__").astype(str)
    
    # Для числовых колонок заполняем NaN
    numeric_cols = [c for c in X.columns if c not in cat_cols and pd.api.types.is_numeric_dtype(X[c])]
    if numeric_cols:
        X[numeric_cols] = X[numeric_cols].fillna(-99999999)
        if test_df is not None:
            # Проверяем, что колонки есть в test_df
            numeric_cols_test = [c for c in numeric_cols if c in test_df.columns]
            if numeric_cols_test:
                test_df[numeric_cols_test] = test_df[numeric_cols_test].fillna(-99999999)
    
    # Финальная проверка размеров
    logger.info(f"Final X shape: {X.shape}")
    if test_df is not None:
        logger.info(f"Final test_df shape: {test_df.shape}")
        
        # КРИТИЧЕСКАЯ ПРОВЕРКА: размер test_df не должен был измениться
        if len(test_df) != original_test_size:
            raise ValueError(
                f"❌ CRITICAL: test_df size changed from {original_test_size} to {len(test_df)} during prepare_all_features! "
                f"This should not happen. Please check the code above."
            )
        
        # Проверяем, что колонки совпадают (кроме возможных различий в категориальных)
        X_cols = set(X.columns)
        test_cols = set(test_df.columns)
        missing_in_test = X_cols - test_cols
        extra_in_test = test_cols - X_cols
        if missing_in_test:
            logger.warning(f"Columns in X but not in test_df: {list(missing_in_test)[:10]}")
        if extra_in_test:
            logger.warning(f"Columns in test_df but not in X: {list(extra_in_test)[:10]}")
    
    return X, test_df, cat_cols

In [53]:
X, test_df, cat_cols = prepare_all_features(X, test_df, CONFIG, logger)

2025-11-18 21:19:39,064 - INFO -  ORIGINAL SIZES: X=9912, test_df=8
INFO:TABULAR_BOOSTING:🔍 ORIGINAL SIZES: X=9912, test_df=8
2025-11-18 21:19:39,067 - INFO -  Generating basic features...
INFO:TABULAR_BOOSTING:🔧 Generating basic features...
2025-11-18 21:19:39,069 - INFO - X shape before basic: (9912, 780)
INFO:TABULAR_BOOSTING:X shape before basic: (9912, 780)
2025-11-18 21:19:39,070 - INFO - test_df shape before basic: (8, 780)
INFO:TABULAR_BOOSTING:test_df shape before basic: (8, 780)
2025-11-18 21:19:39,159 - INFO - X shape after basic: (9912, 780)
INFO:TABULAR_BOOSTING:X shape after basic: (9912, 780)
2025-11-18 21:19:39,160 - INFO - test_df shape after basic: (8, 780)
INFO:TABULAR_BOOSTING:test_df shape after basic: (8, 780)
2025-11-18 21:19:39,162 - INFO -  Generating aggregate features...
INFO:TABULAR_BOOSTING:📊 Generating aggregate features...
2025-11-18 21:19:39,163 - INFO - X shape before aggregate: (9912, 780)
INFO:TABULAR_BOOSTING:X shape before aggregate: (9912, 780)
202

KeyError: "CONFIG['file_names_column']='Id' нет в train"

### Features filter

In [56]:
def filter_features_by_correlation(
    X: pd.DataFrame,
    y: pd.Series,
    config: Dict,
    logger: logging.Logger
) -> Tuple[pd.DataFrame, List[str]]:
    """
    1) Pearson по числовым признакам
    2) phik (если включено и установлен phik)
    Возвращает X с отобранными признаками и список их имён.
    """
    selected = list(X.columns)
    task_type = config["task_type"]

    # --- Pearson ---
    pearson_thr = float(config.get("corr_pearson_min_abs", 0.0) or 0.0)
    if pearson_thr > 0.0:
        logger.info(f"Applying Pearson filter with |corr| >= {pearson_thr}")
        corr_vals = {}
        y_series = pd.Series(y)

        for col in tqdm(selected, desc="Computing Pearson correlations"):
            if pd.api.types.is_numeric_dtype(X[col]):
                try:
                    c = X[col].corr(y_series)
                except Exception:
                    c = np.nan
            else:
                c = np.nan
            corr_vals[col] = c

        corr_series = pd.Series(corr_vals)
        keep_mask = corr_series.abs() < pearson_thr
        kept = corr_series.index[keep_mask].tolist()
        logger.info(
            f"Pearson filter: kept {len(kept)} / {len(corr_series)} features "
            f"(min abs corr {corr_series.abs().min():.6f}, max {corr_series.abs().max():.6f})"
        )
        selected = kept

    # --- phik ---
    use_phik = bool(config.get("corr_use_phik", False))
    phik_thr = float(config.get("corr_phik_min_abs", 0.0) or 0.0)

    if use_phik and phik_thr > 0.0:
        if not HAS_PHIK:
            logger.warning("corr_use_phik=True, но библиотека phik не установлена — пропускаем phik-фильтр.")
        else:
            logger.info(f"Applying phik filter with |phik| >= {phik_thr}")
            df_phik = X[selected].copy()
            tmp_target_col = "__target_for_phik__"
            df_phik[tmp_target_col] = y.values

            # числовые для interval_cols
            interval_cols = [
                c for c in df_phik.columns
                if pd.api.types.is_numeric_dtype(df_phik[c])
            ]
            try:
                phik_matrix = df_phik.phik_matrix(interval_cols=interval_cols)
                target_corr = phik_matrix[tmp_target_col].drop(
                    labels=[tmp_target_col], errors="ignore"
                )
                keep_mask = target_corr.abs() >= phik_thr
                kept_phik = target_corr.index[keep_mask].tolist()
                logger.info(
                    f"phik filter: kept {len(kept_phik)} / {len(target_corr)} features "
                    f"(min abs phik {target_corr.abs().min():.6f}, "
                    f"max {target_corr.abs().max():.6f})"
                )
                # пересечение с уже отфильтрованными по Pearson
                selected = [c for c in selected if c in kept_phik]
            except Exception as e:
                logger.warning(f"phik computation failed, skip phik filter. Error: {e}")

    logger.info(f"Total features after correlation filtering: {len(selected)}")
    return X[selected], selected

In [57]:
# NEW: фильтрация по корреляции
if CONFIG.get("corr_enable", False):
    logger.info("📉 Applying correlation-based feature filtering...")
    # КРИТИЧЕСКАЯ ПРОВЕРКА: сохраняем размер перед фильтрацией
    test_size_before_corr = len(test_df) if test_df is not None else 0
    logger.info(f"🔍 test_df size BEFORE correlation filter: {test_size_before_corr}")
    
    X, selected_cols = filter_features_by_correlation(X, y, CONFIG, logger)

    if test_df is not None:
        missing_in_test = [c for c in selected_cols if c not in test_df.columns]
        if missing_in_test:
            logger.warning(
                f"{len(missing_in_test)} features selected by correlation "
                f"missing in test: {missing_in_test[:10]}..."
            )
        # оставляем только те, что есть в тесте
        keep_for_test = [c for c in selected_cols if c in test_df.columns]
        test_df = test_df[keep_for_test]
        
        # КРИТИЧЕСКАЯ ПРОВЕРКА: размер не должен измениться при фильтрации колонок
        logger.info(f"🔍 test_df size AFTER correlation filter: {len(test_df)}")
        if len(test_df) != test_size_before_corr:
            raise ValueError(
                f"❌ CRITICAL: test_df size changed from {test_size_before_corr} to {len(test_df)} "
                f"during correlation filtering! This should not happen when filtering columns."
            )

In [58]:
 # NEW: sanitize feature names для всех моделей (особенно XGBoost)
logger.info("🧾 Sanitizing feature names...")
# КРИТИЧЕСКАЯ ПРОВЕРКА: сохраняем размер перед sanitize
test_size_before_sanitize = len(test_df) if test_df is not None else 0
logger.info(f"🔍 test_df size BEFORE sanitize: {test_size_before_sanitize}")

X, feature_name_mapping = sanitize_feature_names(X, logger)

if test_df is not None:
    # сначала переименуем колонки теста по тому же маппингу
    test_df = test_df.rename(columns=feature_name_mapping)
    # если в тесте остались какие-то дополнительные колонки (в реальности редко),
    # можно ещё раз прогнать sanitize, но имена уже будут безопасные
    # test, _ = sanitize_feature_names(test, logger)
    
    # КРИТИЧЕСКАЯ ПРОВЕРКА: размер не должен измениться при переименовании
    logger.info(f"🔍 test_df size AFTER sanitize: {len(test_df)}")
    if len(test_df) != test_size_before_sanitize:
        raise ValueError(
            f"❌ CRITICAL: test_df size changed from {test_size_before_sanitize} to {len(test_df)} "
            f"during sanitize! This should not happen when renaming columns."
        )

# обновляем список категориальных колонок под новые имена
cat_cols = [feature_name_mapping.get(c, c) for c in cat_cols]

2025-11-18 21:21:12,945 - INFO -  Sanitizing feature names...
INFO:TABULAR_BOOSTING:🧾 Sanitizing feature names...
2025-11-18 21:21:12,948 - INFO -  test_df size BEFORE sanitize: 8
INFO:TABULAR_BOOSTING:🔍 test_df size BEFORE sanitize: 8
2025-11-18 21:21:13,048 - INFO -  test_df size AFTER sanitize: 8
INFO:TABULAR_BOOSTING:🔍 test_df size AFTER sanitize: 8


### Cross-validation

In [59]:
def make_folds(
    y: pd.Series,
    config: Dict
):
    n_splits = config["cv_n_splits"]
    seed = config["cv_random_state"]
    shuffle = config["cv_shuffle"]
    stratified = config["cv_stratified"]
    task_type = config["task_type"]

    if stratified and task_type in ["binary", "multiclass"]:
        splitter = StratifiedKFold(
            n_splits=n_splits,
            shuffle=shuffle,
            random_state=seed
        )
    else:
        splitter = KFold(
            n_splits=n_splits,
            shuffle=shuffle,
            random_state=seed
        )
    return list(splitter.split(np.zeros(len(y)), y))


def build_catboost_model(
    config: Dict
):
    params = config["catboost_params"].copy()
    
    # Добавляем device в параметры (CatBoost использует "GPU" или "CPU")
    device_str = config.get("device", "CPU")
    if device_str.upper() == "CUDA":
        params["task_type"] = "GPU"
    elif device_str.upper() == "GPU":
        params["task_type"] = "GPU"
    else:
        params["task_type"] = "CPU"
    
    if config["task_type"] == "regression":
        params["loss_function"] = "RMSE"
        params["eval_metric"] = "RMSE"
        model = CatBoostRegressor(**params)
    else:
        if config["task_type"] == "multiclass":
            params["loss_function"] = "MultiClass"
            params["eval_metric"] = "MultiClass"
        model = CatBoostClassifier(**params)
    return model


def build_lgb_params(
    config: Dict,
    num_classes: Optional[int] = None
):
    params = config["lgb_params"].copy()
    task_type = config["task_type"]

    # ---------- Device (CPU / GPU) ----------
    device_str = config.get("device", "CPU").upper()
    if device_str in ["CUDA", "GPU"]:
        # для LightGBM правильно "device_type"
        params["device_type"] = "gpu"
    else:
        params["device_type"] = "cpu"

    # ---------- Objective + metric под тип задачи ----------
    if task_type == "regression":
        # обычная регрессия
        params["objective"] = "regression"
        # метрики LightGBM задаются ключом "metric"
        params["metric"] = ["mae"]   # или ["rmse"]
    elif task_type == "binary":
        params["objective"] = "binary"
        params["metric"] = ["auc", "binary_logloss"]
    else:  # multiclass
        if num_classes is None or num_classes <= 1:
            raise ValueError("num_classes must be > 1 for multiclass")
        params["objective"] = "multiclass"
        params["num_class"] = num_classes
        params["metric"] = ["multi_logloss", "multi_error"]

    return params



def build_xgb_params(
    config: Dict,
    num_classes: Optional[int] = None,
):
    params = config["xgb_params"].copy()
    task_type = config["task_type"]

    # ----- чистим возможный мусор -----
    # если в конфиге было что-то вроде "device": 0 — убираем
    if "device" in params and not isinstance(params["device"], str):
        params.pop("device")

    # ----- CPU / GPU -----
    device_str = config.get("device", "CPU").upper()
    if device_str in ["CUDA", "GPU"]:
        # XGBoost 2.x хочет строковый device
        params["device"] = "cuda"      # можно "cuda:0"
        # на GPU обычно достаточно tree_method="hist" или "gpu_hist"
        # оставим "gpu_hist", если в конфиге уже задан, иначе поставим мы
        params.setdefault("tree_method", "gpu_hist")
    else:
        params["device"] = "cpu"
        params.setdefault("tree_method", "hist")

    # потоков по умолчанию — все ядра
    params.setdefault("nthread", 0)

    # ----- тип задачи -----
    if task_type == "regression":
        params["objective"] = "reg:squarederror"
        params["eval_metric"] = "mae"           # можно поменять на "rmse"
    elif task_type == "binary":
        params["objective"] = "binary:logistic"
        params["eval_metric"] = "auc"
    else:
        # multiclass
        if num_classes is None:
            raise ValueError("num_classes must be provided for multiclass")
        params["objective"] = "multi:softprob"
        params["num_class"] = num_classes
        params["eval_metric"] = "mlogloss"

    return params




def _evaluate_predictions(
    y_true: pd.Series,
    preds: np.ndarray,
    task_type: str,
    name_prefix: str,
    logger: logging.Logger
) -> Dict[str, float]:
    metrics = {}
    if preds is None:
        return metrics

    if task_type == "regression":
        rmse = mean_squared_error(y_true, preds, squared=False)
        mae = mean_absolute_error(y_true, preds)
        metrics[f"{name_prefix}_RMSE"] = rmse
        metrics[f"{name_prefix}_MAE"] = rmse
        
        logger.info(f"{name_prefix} RMSE: {rmse:.6f} MAE: {mae:.6f}")
    elif task_type == "binary":
        auc = roc_auc_score(y_true, preds)
        logloss = log_loss(y_true, preds)

        ## можно сделать подбор оптимального порога
        preds_label = (preds >= 0.5).astype(int)
        acc = accuracy_score(y_true, preds_label)
        cm = confusion_matrix(y_true, preds_label)
        metrics[f"{name_prefix}_AUC"] = auc
        metrics[f"{name_prefix}_LogLoss"] = logloss
        metrics[f"{name_prefix}_ACC"] = acc
        logger.info(f"{name_prefix} AUC: {auc:.6f}, LogLoss: {logloss:.6f}, ACC: {acc:.6f}")
        logger.info(f"{name_prefix} confusion matrix:\n{cm}")
    else:
        # multiclass
        # если пришли вероятности (n_samples, n_classes) — берём argmax
        if preds.ndim == 2:
            preds_label = np.argmax(preds, axis=1)
        else:
            preds_label = preds

        acc = accuracy_score(y_true, preds_label)
        cm = confusion_matrix(y_true, preds_label)
        metrics[f"{name_prefix}_ACC"] = acc
        logger.info(f"{name_prefix} ACC: {acc:.6f}")
        logger.info(f"{name_prefix} confusion matrix:\n{cm}")

    return metrics


def train_and_evaluate(
    X: pd.DataFrame,
    y: pd.Series,
    cat_cols: List[str],
    config: Dict,
    logger: logging.Logger
) -> Dict:
    task_type = config["task_type"]
    folds = make_folds(y, config)
    logger.info(f"Using {len(folds)} folds.")

    cat_features_idx = [X.columns.get_loc(c) for c in cat_cols if c in X.columns]

    # СНАЧАЛА считаем classes_
    classes_ = np.unique(y) if task_type == "multiclass" else None

    n = len(y)
    if task_type == "multiclass":
        n_classes = len(classes_)
        oof_preds_cat = np.zeros((n, n_classes)) if config["use_catboost"] else None
        oof_preds_lgb = np.zeros((n, n_classes)) if config["use_lightgbm"] else None
        oof_preds_xgb = np.zeros((n, n_classes)) if config["use_xgboost"] else None
    else:
        oof_preds_cat = np.zeros(n) if config["use_catboost"] else None
        oof_preds_lgb = np.zeros(n) if config["use_lightgbm"] else None
        oof_preds_xgb = np.zeros(n) if config["use_xgboost"] else None

    models_cat: List = []
    models_lgb: List = []
    models_xgb: List = []

    feature_importances_lgb = []
    feature_importances_xgb = []

    for fold_idx, (tr_idx, val_idx) in enumerate(tqdm(folds, desc="Cross-validation folds")):
        logger.info(f"========== Fold {fold_idx + 1}/{len(folds)} ==========")

        X_tr, X_val = X.iloc[tr_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[tr_idx], y.iloc[val_idx]

        if config["use_catboost"]:
            logger.info("Training CatBoost...")
            train_pool = Pool(
                X_tr, y_tr,
                cat_features=cat_features_idx
            )
            val_pool = Pool(
                X_val, y_val,
                cat_features=cat_features_idx
            )
            model_cat = build_catboost_model(config)
            # Early stopping для CatBoost
            early_stopping_rounds = config.get("early_stopping_rounds", 100)
            model_cat.fit(
                train_pool, 
                eval_set=val_pool,
                early_stopping_rounds=early_stopping_rounds,
                use_best_model=True
            )

            if task_type == "regression":
                preds_val = model_cat.predict(val_pool)
            else:
                proba = model_cat.predict_proba(val_pool)
                if task_type == "binary":
                    preds_val = proba[:, 1]
                else:
                    oof_preds_cat[val_idx, :] = proba
                    preds_val = proba  # для единообразия, если вдруг где-то используешь

            if task_type != "multiclass":
                oof_preds_cat[val_idx] = preds_val
            models_cat.append(model_cat)

        if config["use_lightgbm"]:
            logger.info("Training LightGBM...")

            num_classes = len(classes_) if task_type == "multiclass" else None
            lgb_params = build_lgb_params(config, num_classes=num_classes)

            X_tr_lgb = X_tr.copy()
            X_val_lgb = X_val.copy()
            for c in cat_cols:
                if c in X_tr_lgb.columns:
                    X_tr_lgb[c] = X_tr_lgb[c].astype("category")
                    X_val_lgb[c] = X_val_lgb[c].astype("category")

            lgb_train = lgb.Dataset(X_tr_lgb, label=y_tr)
            lgb_val = lgb.Dataset(X_val_lgb, label=y_val)
            print(lgb_params)
            # Early stopping для LightGBM
            early_stopping_rounds = config.get("early_stopping_rounds", 100)
            model_lgb = lgb.train(
                lgb_params,
                lgb_train,
                num_boost_round=1000,
                valid_sets=[lgb_train, lgb_val],
                valid_names=["train", "valid"],
                callbacks=[
                    lgb.early_stopping(stopping_rounds=early_stopping_rounds),
                    lgb.log_evaluation(100)
                ]
            )

            if task_type == "regression":
                preds_val = model_lgb.predict(X_val_lgb, num_iteration=model_lgb.best_iteration)
            else:
                proba = model_lgb.predict(X_val_lgb, num_iteration=model_lgb.best_iteration)
                if task_type == "binary":
                    preds_val = proba
                else:
                    # multiclass: сохраняем вероятности
                    oof_preds_lgb[val_idx, :] = proba
                    preds_val = proba

            if task_type != "multiclass":
                oof_preds_lgb[val_idx] = preds_val
            models_lgb.append(model_lgb)
            feature_importances_lgb.append(model_lgb.feature_importance(importance_type="gain"))

        if config["use_xgboost"]:
            logger.info("Training XGBoost...")
            num_classes = len(classes_) if task_type == "multiclass" else None
            xgb_params = build_xgb_params(config, num_classes=num_classes)

            X_tr_xgb = X_tr.copy()
            X_val_xgb = X_val.copy()
            for c in X_tr_xgb.columns:
                if X_tr_xgb[c].dtype == "object":
                    X_tr_xgb[c] = X_tr_xgb[c].astype("category")
                    X_val_xgb[c] = X_val_xgb[c].astype("category")

            dtrain = xgb.DMatrix(X_tr_xgb, label=y_tr, enable_categorical=True)
            dvalid = xgb.DMatrix(X_val_xgb, label=y_val, enable_categorical=True)

            evals = [(dtrain, "train"), (dvalid, "valid")]
            print(dtrain, dvalid)
            print(xgb_params)
            # Early stopping для XGBoost
            early_stopping_rounds = config.get("early_stopping_rounds", 100)
            model_xgb = xgb.train(
                params=xgb_params,
                dtrain=dtrain,
                num_boost_round=1000,
                evals=evals,
                early_stopping_rounds=early_stopping_rounds,
                verbose_eval=100
            )

            if task_type == "regression":
                preds_val = model_xgb.predict(
                    dvalid,
                    iteration_range=(0, model_xgb.best_iteration + 1)
                )
            else:
                proba = model_xgb.predict(
                    dvalid,
                    iteration_range=(0, model_xgb.best_iteration + 1)
                )
                if task_type == "binary":
                    preds_val = proba
                else:
                    # multiclass: сохраняем вероятности
                    oof_preds_xgb[val_idx, :] = proba
                    preds_val = proba

            if task_type != "multiclass":
                oof_preds_xgb[val_idx] = preds_val
            models_xgb.append(model_xgb)

            fi_xgb = model_xgb.get_score(importance_type="gain")
            fi_vec = np.array([fi_xgb.get(f, 0.0) for f in X.columns])
            feature_importances_xgb.append(fi_vec)

        gc.collect()

    metrics_all: Dict[str, float] = {}
    metrics_all.update(_evaluate_predictions(y, oof_preds_cat, task_type, "CatBoost", logger))
    metrics_all.update(_evaluate_predictions(y, oof_preds_lgb, task_type, "LightGBM", logger))
    metrics_all.update(_evaluate_predictions(y, oof_preds_xgb, task_type, "XGBoost", logger))

    logger.info("Evaluating blended OOF predictions...")
    blend_weights = config["blend_weights"]
    
    blend_preds = None
    if task_type == "multiclass":
        # определяем число классов по первой не-None матрице
        num_classes = None
        for arr in [oof_preds_cat, oof_preds_lgb, oof_preds_xgb]:
            if arr is not None:
                num_classes = arr.shape[1]
                break
        if num_classes is None:
            logger.warning("No OOF predictions for multiclass blend.")
        else:
            blend_num = np.zeros((n, num_classes), dtype=float)
            blend_den = np.zeros(n, dtype=float)

            if oof_preds_cat is not None and blend_weights.get("catboost", 0) != 0:
                w = blend_weights["catboost"]
                blend_num += w * oof_preds_cat
                blend_den += w
            if oof_preds_lgb is not None and blend_weights.get("lightgbm", 0) != 0:
                w = blend_weights["lightgbm"]
                blend_num += w * oof_preds_lgb
                blend_den += w
            if oof_preds_xgb is not None and blend_weights.get("xgboost", 0) != 0:
                w = blend_weights["xgboost"]
                blend_num += w * oof_preds_xgb
                blend_den += w

            valid_mask = blend_den > 0
            if valid_mask.any():
                blend_preds = np.zeros_like(blend_num)
                # нормируем по весам, broadcast по axis=1
                blend_preds[valid_mask] = (
                    blend_num[valid_mask] /
                    blend_den[valid_mask][:, None]
                )
                metrics_all.update(_evaluate_predictions(y, blend_preds, task_type, "Blend", logger))
            else:
                logger.warning("No models contributions to blend; blend_den is zero everywhere.")
    else:
        # binary / regression — как раньше (1D)
        blend_num = np.zeros(n, dtype=float)
        blend_den = np.zeros(n, dtype=float)

        if oof_preds_cat is not None and blend_weights.get("catboost", 0) != 0:
            w = blend_weights["catboost"]
            blend_num += w * oof_preds_cat
            blend_den += w
        if oof_preds_lgb is not None and blend_weights.get("lightgbm", 0) != 0:
            w = blend_weights["lightgbm"]
            blend_num += w * oof_preds_lgb
            blend_den += w
        if oof_preds_xgb is not None and blend_weights.get("xgboost", 0) != 0:
            w = blend_weights["xgboost"]
            blend_num += w * oof_preds_xgb
            blend_den += w

        valid_mask = blend_den > 0
        if valid_mask.any():
            blend_preds = np.zeros_like(blend_num)
            blend_preds[valid_mask] = blend_num[valid_mask] / blend_den[valid_mask]
            metrics_all.update(_evaluate_predictions(y, blend_preds, task_type, "Blend", logger))
        else:
            logger.warning("No models contributions to blend; blend_den is zero everywhere.")
    logger.info("========== OOF metrics ==========")
    logger.info(metrics_all)

    results = {
        "oof_catboost": oof_preds_cat,
        "oof_lightgbm": oof_preds_lgb,
        "oof_xgboost": oof_preds_xgb,
        "oof_blend": blend_preds,
        "metrics": metrics_all,
        "models_catboost": models_cat,
        "models_lightgbm": models_lgb,
        "models_xgboost": models_xgb
    }

    if feature_importances_lgb:
        fi_lgb = np.mean(feature_importances_lgb, axis=0)
        results["feature_importance_lgb"] = fi_lgb

    if feature_importances_xgb:
        fi_xgb_mean = np.mean(feature_importances_xgb, axis=0)
        results["feature_importance_xgb"] = fi_xgb_mean

    return results


def ensemble_predict(
    models: Dict[str, List],
    X: pd.DataFrame,
    cat_cols: List[str],
    config: Dict
) -> Dict[str, Optional[np.ndarray]]:
    """
    Делает предсказания ансамблем:
      - по каждому бустингу усредняет предсказания по фолдам
      - формирует blended предсказания по весам

    Возвращает:
      {
        "catboost": np.ndarray или None,
        "lightgbm": np.ndarray или None,
        "xgboost": np.ndarray или None,
        "blend": np.ndarray или None,
      }
    """
    task_type = config["task_type"]
    blend_weights = config["blend_weights"]

    n = len(X)
    preds_cat = None
    preds_lgb = None
    preds_xgb = None

    X_lgb = X.copy()
    for c in cat_cols:
        if c in X_lgb.columns:
            X_lgb[c] = X_lgb[c].astype("category")

    # CatBoost
    if models.get("catboost"):
        cat_features_idx = [X.columns.get_loc(c) for c in cat_cols if c in X.columns]
        test_pool = Pool(X, cat_features=cat_features_idx)
        for m in models["catboost"]:
            if task_type == "regression":
                proba = m.predict(test_pool)              # (n,)
            else:
                proba_full = m.predict_proba(test_pool)   # (n,2) или (n,C)
                if task_type == "binary":
                    proba = proba_full[:, 1]              # (n,)
                else:
                    proba = proba_full                    # (n,C)
            if preds_cat is None:
                preds_cat = np.zeros_like(proba, dtype=float)
            preds_cat += proba
        preds_cat /= len(models["catboost"])

    if models.get("lightgbm"):
        for m in models["lightgbm"]:
            if task_type == "regression":
                proba = m.predict(X_lgb, num_iteration=m.best_iteration)  # (n,)
            else:
                proba_full = m.predict(X_lgb, num_iteration=m.best_iteration)  # (n,) или (n,C)
                if task_type == "binary":
                    proba = proba_full                                      # (n,)
                else:
                    proba = proba_full                                      # (n,C)
            if preds_lgb is None:
                preds_lgb = np.zeros_like(proba, dtype=float)
            preds_lgb += proba
        preds_lgb /= len(models["lightgbm"])

    # XGBoost
    if models.get("xgboost"):
        X_xgb = X.copy()
        for c in X_xgb.columns:
            if X_xgb[c].dtype == "object":
                X_xgb[c] = X_xgb[c].astype("category")
        dtest = xgb.DMatrix(X_xgb, enable_categorical=True)

        for m in models["xgboost"]:
            if task_type == "regression":
                proba = m.predict(
                    dtest,
                    iteration_range=(0, m.best_iteration + 1)
                )  # shape: (n,)
            else:
                proba_full = m.predict(
                    dtest,
                    iteration_range=(0, m.best_iteration + 1)
                )  # shape: (n,) for binary, (n, C) for multiclass
                if task_type == "binary":
                    proba = proba_full          # (n,)
                else:
                    proba = proba_full          # (n, C)

            if preds_xgb is None:
                preds_xgb = np.zeros_like(proba, dtype=float)
            preds_xgb += proba

        preds_xgb /= len(models["xgboost"])

    # Blending
    blend_preds = None
    if task_type == "multiclass":
        # определяем форму по первой не-None матрице
        base = None
        for arr in [preds_cat, preds_lgb, preds_xgb]:
            if arr is not None:
                base = arr
                break
        if base is not None:
            blend_num = np.zeros_like(base, dtype=float)  # (n,C)
            blend_den = np.zeros(n, dtype=float)

            if preds_cat is not None and blend_weights.get("catboost", 0) != 0:
                w = blend_weights["catboost"]
                blend_num += w * preds_cat
                blend_den += w
            if preds_lgb is not None and blend_weights.get("lightgbm", 0) != 0:
                w = blend_weights["lightgbm"]
                blend_num += w * preds_lgb
                blend_den += w
            if preds_xgb is not None and blend_weights.get("xgboost", 0) != 0:
                w = blend_weights["xgboost"]
                blend_num += w * preds_xgb
                blend_den += w

            valid_mask = blend_den > 0
            if valid_mask.any():
                blend_preds = np.zeros_like(blend_num)
                blend_preds[valid_mask] = (
                    blend_num[valid_mask] /
                    blend_den[valid_mask][:, None]
                )
    else:
        blend_num = np.zeros(n, dtype=float)
        blend_den = np.zeros(n, dtype=float)
        if preds_cat is not None and blend_weights.get("catboost", 0) != 0:
            w = blend_weights["catboost"]
            blend_num += w * preds_cat
            blend_den += w
        if preds_lgb is not None and blend_weights.get("lightgbm", 0) != 0:
            w = blend_weights["lightgbm"]
            blend_num += w * preds_lgb
            blend_den += w
        if preds_xgb is not None and blend_weights.get("xgboost", 0) != 0:
            w = blend_weights["xgboost"]
            blend_num += w * preds_xgb
            blend_den += w

        valid_mask = blend_den > 0
        if valid_mask.any():
            blend_preds = np.zeros_like(blend_num)
            blend_preds[valid_mask] = blend_num[valid_mask] / blend_den[valid_mask]

    return {
        "catboost": preds_cat,
        "lightgbm": preds_lgb,
        "xgboost": preds_xgb,
        "blend": blend_preds
    }


In [60]:
# Удаляем ID только перед обучением бустингов,
# чтобы ID не утек в модели, но остался для генерации фич (картинки/тексты и т.п.)
id_col = CONFIG["id_column"]

if id_col in X.columns:
    logger.info(f"Dropping ID column '{id_col}' before boosting")
    X = X.drop(columns=[id_col])

if test_df is not None and id_col in test_df.columns:
    test_df = test_df.drop(columns=[id_col])

# На всякий случай исключаем ID из списка категориальных колонок
if "cat_cols" in globals():
    cat_cols = [c for c in cat_cols if c != id_col]

results = train_and_evaluate(X, y, cat_cols, CONFIG, logger)

# ----- Save OOF, metrics, FI -----
logger.info("💾 Saving artifacts...")
if results["oof_catboost"] is not None:
    np.save(os.path.join(output_dir, "oof_catboost.npy"), results["oof_catboost"])
if results["oof_lightgbm"] is not None:
    np.save(os.path.join(output_dir, "oof_lightgbm.npy"), results["oof_lightgbm"])
if results["oof_xgboost"] is not None:
    np.save(os.path.join(output_dir, "oof_xgboost.npy"), results["oof_xgboost"])
if results["oof_blend"] is not None:
    np.save(os.path.join(output_dir, "oof_blend.npy"), results["oof_blend"])

with open(os.path.join(output_dir, "metrics.json"), "w", encoding="utf-8") as f:
    json.dump(results["metrics"], f, indent=2, ensure_ascii=False)


2025-11-18 21:21:27,345 - INFO - Using 5 folds.
INFO:TABULAR_BOOSTING:Using 5 folds.
Cross-validation folds:   0%|          | 0/5 [00:00<?, ?it/s]2025-11-18 21:21:27,349 - INFO - ========== Fold 1/5 ==========
INFO:TABULAR_BOOSTING:========== Fold 1/5 ==========
2025-11-18 21:21:27,409 - INFO - Training CatBoost...
INFO:TABULAR_BOOSTING:Training CatBoost...


0:	learn: 20.3802004	test: 20.9300699	best: 20.9300699 (0)	total: 175ms	remaining: 10m 12s
100:	learn: 16.9138527	test: 18.2101740	best: 18.2101740 (100)	total: 3.44s	remaining: 1m 55s
200:	learn: 15.8099983	test: 17.9217795	best: 17.9217795 (200)	total: 6.48s	remaining: 1m 46s
300:	learn: 14.9926065	test: 17.8779569	best: 17.8674114 (288)	total: 9.46s	remaining: 1m 40s
400:	learn: 14.2649801	test: 17.8180804	best: 17.8180804 (400)	total: 12.4s	remaining: 1m 36s
500:	learn: 13.6613756	test: 17.8031024	best: 17.8031024 (500)	total: 15.4s	remaining: 1m 32s
600:	learn: 13.0777474	test: 17.7944839	best: 17.7931555 (555)	total: 18.4s	remaining: 1m 28s
700:	learn: 12.5647572	test: 17.7754482	best: 17.7710601 (661)	total: 21.4s	remaining: 1m 25s
800:	learn: 12.1154208	test: 17.7859340	best: 17.7701902 (711)	total: 24.4s	remaining: 1m 22s
bestTest = 17.77019016
bestIteration = 711
Shrink model to first 712 iterations.


Cross-validation folds:  20%|██        | 1/5 [00:26<01:47, 26.76s/it]2025-11-18 21:21:54,109 - INFO - ========== Fold 2/5 ==========
INFO:TABULAR_BOOSTING:========== Fold 2/5 ==========
2025-11-18 21:21:54,141 - INFO - Training CatBoost...
INFO:TABULAR_BOOSTING:Training CatBoost...


0:	learn: 20.4785146	test: 20.4554206	best: 20.4554206 (0)	total: 32.6ms	remaining: 1m 54s
100:	learn: 16.7732724	test: 18.3915519	best: 18.3915519 (100)	total: 3.29s	remaining: 1m 50s
200:	learn: 15.7253562	test: 18.2091892	best: 18.2049887 (193)	total: 6.35s	remaining: 1m 44s
300:	learn: 14.8481892	test: 18.1212907	best: 18.1212907 (300)	total: 9.36s	remaining: 1m 39s
400:	learn: 14.1574546	test: 18.0780146	best: 18.0780146 (400)	total: 12.3s	remaining: 1m 35s
500:	learn: 13.5480314	test: 18.0440833	best: 18.0440833 (500)	total: 15.3s	remaining: 1m 31s
600:	learn: 13.0442520	test: 18.0214352	best: 18.0166609 (592)	total: 18.2s	remaining: 1m 27s
bestTest = 18.01666088
bestIteration = 592
Shrink model to first 593 iterations.


Cross-validation folds:  40%|████      | 2/5 [00:49<01:13, 24.34s/it]2025-11-18 21:22:16,752 - INFO - ========== Fold 3/5 ==========
INFO:TABULAR_BOOSTING:========== Fold 3/5 ==========
2025-11-18 21:22:16,781 - INFO - Training CatBoost...
INFO:TABULAR_BOOSTING:Training CatBoost...


...

Cross-validation folds:  60%|██████    | 3/5 [01:15<00:50, 25.16s/it]2025-11-18 21:22:42,895 - INFO - ========== Fold 4/5 ==========
INFO:TABULAR_BOOSTING:========== Fold 4/5 ==========
2025-11-18 21:22:42,931 - INFO - Training CatBoost...
INFO:TABULAR_BOOSTING:Training CatBoost...


0:	learn: 20.4585323	test: 20.4690535	best: 20.4690535 (0)	total: 34.7ms	remaining: 2m 1s
100:	learn: 16.8805237	test: 18.1264409	best: 18.1264409 (100)	total: 3.18s	remaining: 1m 47s
200:	learn: 15.8272461	test: 17.8878863	best: 17.8878863 (200)	total: 6.21s	remaining: 1m 41s
300:	learn: 14.9553950	test: 17.8225005	best: 17.8109696 (282)	total: 9.19s	remaining: 1m 37s
400:	learn: 14.1952750	test: 17.7575274	best: 17.7575274 (400)	total: 12.2s	remaining: 1m 34s
500:	learn: 13.5452484	test: 17.7425565	best: 17.7310216 (450)	total: 15.2s	remaining: 1m 30s
600:	learn: 13.0230631	test: 17.7235745	best: 17.7211040 (595)	total: 18.1s	remaining: 1m 27s
700:	learn: 12.5094466	test: 17.7015735	best: 17.6963550 (693)	total: 21.1s	remaining: 1m 24s
800:	learn: 12.0392505	test: 17.6709593	best: 17.6694870 (796)	total: 24.1s	remaining: 1m 21s
900:	learn: 11.6328813	test: 17.6516994	best: 17.6502952 (898)	total: 27.1s	remaining: 1m 18s
1000:	learn: 11.2662990	test: 17.6421161	best: 17.6421161 (1000)

Cross-validation folds:  80%|████████  | 4/5 [01:52<00:29, 29.71s/it]2025-11-18 21:23:19,569 - INFO - ========== Fold 5/5 ==========
INFO:TABULAR_BOOSTING:========== Fold 5/5 ==========
2025-11-18 21:23:19,602 - INFO - Training CatBoost...
INFO:TABULAR_BOOSTING:Training CatBoost...


0:	learn: 20.4366613	test: 20.5950846	best: 20.5950846 (0)	total: 32.5ms	remaining: 1m 53s
100:	learn: 16.8436958	test: 18.1491553	best: 18.1491553 (100)	total: 3.25s	remaining: 1m 49s
200:	learn: 15.7681799	test: 17.9643999	best: 17.9589224 (199)	total: 6.31s	remaining: 1m 43s
300:	learn: 14.9546672	test: 17.8921413	best: 17.8921413 (300)	total: 9.31s	remaining: 1m 38s
400:	learn: 14.1890763	test: 17.8643958	best: 17.8604943 (395)	total: 12.3s	remaining: 1m 35s
500:	learn: 13.5733099	test: 17.8742912	best: 17.8584594 (447)	total: 15.3s	remaining: 1m 31s
bestTest = 17.85845937
bestIteration = 447
Shrink model to first 448 iterations.


Cross-validation folds: 100%|██████████| 5/5 [02:10<00:00, 26.10s/it]
2025-11-18 21:23:37,873 - INFO - CatBoost RMSE: 17.715399 MAE: 12.937462
INFO:TABULAR_BOOSTING:CatBoost RMSE: 17.715399 MAE: 12.937462
2025-11-18 21:23:37,875 - INFO - Evaluating blended OOF predictions...
INFO:TABULAR_BOOSTING:Evaluating blended OOF predictions...
2025-11-18 21:23:37,881 - INFO - Blend RMSE: 17.715399 MAE: 12.937462
INFO:TABULAR_BOOSTING:Blend RMSE: 17.715399 MAE: 12.937462
2025-11-18 21:23:37,883 - INFO - ========== OOF metrics ==========
INFO:TABULAR_BOOSTING:========== OOF metrics ==========
2025-11-18 21:23:37,885 - INFO - {'CatBoost_RMSE': 17.715398906717056, 'CatBoost_MAE': 17.715398906717056, 'Blend_RMSE': 17.715398906717056, 'Blend_MAE': 17.715398906717056}
INFO:TABULAR_BOOSTING:{'CatBoost_RMSE': 17.715398906717056, 'CatBoost_MAE': 17.715398906717056, 'Blend_RMSE': 17.715398906717056, 'Blend_MAE': 17.715398906717056}
2025-11-18 21:23:37,892 - INFO -  Saving artifacts...
INFO:TABULAR_BOOSTING

### Importances

In [61]:
if "feature_importance_lgb" in results:
    fi = results["feature_importance_lgb"]
    fi_df = pd.DataFrame({
        "feature": X.columns,
        "importance_gain": fi
    }).sort_values("importance_gain", ascending=False)
    fi_df.to_csv(os.path.join(output_dir, "feature_importance_lgb.csv"), index=False)
    logger.info("Top features (LGB, gain):")
    logger.info(fi_df.head(CONFIG["top_features_to_show"]))

if "feature_importance_xgb" in results:
    fi_xgb = results["feature_importance_xgb"]
    fi_xgb_df = pd.DataFrame({
        "feature": X.columns,
        "importance_gain": fi_xgb
    }).sort_values("importance_gain", ascending=False)
    fi_xgb_df.to_csv(os.path.join(output_dir, "feature_importance_xgb.csv"), index=False)
    logger.info("Top features (XGB, gain):")
    logger.info(fi_xgb_df.head(CONFIG["top_features_to_show"]))

### Submission

In [62]:
# ----- Test predictions (ensemble) -----
if test_df is not None:
    # NEW: выравниваем категориальные значения теста по train,
    # маппим все unseen → "other" (если она есть), иначе → мода трейна
    if cat_cols:
        logger.info("🧩 Aligning categorical values in test with train for XGBoost...")
        for c in cat_cols:
            if c in test_df.columns and c in X.columns:

                train_vals = X[c].dropna()
                if train_vals.empty:
                    continue

                known = set(train_vals.unique())
                mask_new = ~test_df[c].isin(known)

                if mask_new.any():
                    # проверяем существование категорий "other"
                    known_lower = {str(v).lower() for v in known}
                    if "other" in known_lower:
                        # находим точное значение "other" в train
                        other_value = next(v for v in known if str(v).lower() == "other")
                        replacement = other_value
                    else:
                        # если "other" нет — fallback на моду
                        replacement = train_vals.mode(dropna=True).iloc[0]

                    logger.info(
                        f"🔄 Column '{c}': mapping {mask_new.sum()} unseen categories → '{replacement}'"
                    )
                    test_df.loc[mask_new, c] = replacement

    logger.info("📤 Making ensemble predictions for test...")
    # КРИТИЧЕСКАЯ ПРОВЕРКА: убеждаемся, что test_df имеет правильный размер
    logger.info(f"test_df shape before ensemble_predict: {test_df.shape}")
    logger.info(f"test_ids length: {len(test_ids) if test_ids is not None else 'None'}")
    
    # КРИТИЧЕСКАЯ ПРОВЕРКА: если размер test_df не совпадает с test_ids, это ошибка
    if test_ids is not None:
        expected_test_size = len(test_ids)
        actual_test_size = len(test_df)
        if actual_test_size != expected_test_size:
            error_msg = (
                f"❌ CRITICAL ERROR: test_df size ({actual_test_size}) != test_ids size ({expected_test_size})! "
                f"This indicates a problem in data processing. "
                f"test_df should have {expected_test_size} rows, but has {actual_test_size}. "
                f"Please check your data processing pipeline, especially generate_aggregate_features."
            )
            logger.error(error_msg)
            raise ValueError(error_msg)
    
    pred_dict = ensemble_predict(
        models={
            "catboost": results["models_catboost"],
            "lightgbm": results["models_lightgbm"],
            "xgboost": results["models_xgboost"],
        },
        X=test_df,
        cat_cols=cat_cols,
        config=CONFIG
    )

    if test_ids is None:
        test_ids_series = pd.Series(np.arange(len(test_df)), name=id_col)
    else:
        test_ids_series = test_ids

    # Проверка размеров перед созданием submission
    logger.info(f"Test DataFrame shape: {test_df.shape}")
    logger.info(f"Test IDs length: {len(test_ids_series)}")

    for name, preds in pred_dict.items():
        if preds is None:
            continue

        # Проверка размеров предсказаний
        if isinstance(preds, np.ndarray):
            pred_len = preds.shape[0] if preds.ndim == 1 else preds.shape[0]
            logger.info(f"Predictions '{name}' shape: {preds.shape}, length: {pred_len}")
            
            if pred_len != len(test_df):
                raise ValueError(
                    f"Predictions length {pred_len} != test_df length {len(test_df)} for {name}. "
                    f"Test_df shape: {test_df.shape}, predictions shape: {preds.shape}"
                )

        if isinstance(preds, np.ndarray) and preds.ndim == 2:
            n_classes = preds.shape[1]
            data = {
                id_col: test_ids_series.values,  # Используем .values для гарантии правильного размера
                "prediction": np.argmax(preds, axis=1),  # предсказанный класс
            }
            for k in range(n_classes):
                data[f"proba_class_{k}"] = preds[:, k]
            out_df = pd.DataFrame(data)
        else:
            # Убеждаемся, что размеры совпадают
            preds_flat = preds.flatten() if isinstance(preds, np.ndarray) else preds
            if len(preds_flat) != len(test_ids_series):
                raise ValueError(
                    f"Predictions length {len(preds_flat)} != test_ids length {len(test_ids_series)} for {name}"
                )
            out_df = pd.DataFrame({
                id_col: test_ids_series.values,  # Используем .values
                "prediction": preds_flat
            })

        out_df.to_csv(os.path.join(output_dir, f"pred_{name}.csv"), index=False)
        logger.info(f"Saved predictions: pred_{name}.csv")

elapsed = time.time() - start_time
logger.info(f"⏱ Finished in {elapsed:.1f} seconds")

2025-11-18 21:23:58,013 - INFO -  Aligning categorical values in test with train for XGBoost...
INFO:TABULAR_BOOSTING:🧩 Aligning categorical values in test with train for XGBoost...
2025-11-18 21:23:58,040 - INFO -  Making ensemble predictions for test...
INFO:TABULAR_BOOSTING:📤 Making ensemble predictions for test...
2025-11-18 21:23:58,042 - INFO - test_df shape before ensemble_predict: (8, 780)
INFO:TABULAR_BOOSTING:test_df shape before ensemble_predict: (8, 780)
2025-11-18 21:23:58,044 - INFO - test_ids length: 8
INFO:TABULAR_BOOSTING:test_ids length: 8
2025-11-18 21:23:58,103 - INFO - Test DataFrame shape: (8, 780)
INFO:TABULAR_BOOSTING:Test DataFrame shape: (8, 780)
2025-11-18 21:23:58,105 - INFO - Test IDs length: 8
INFO:TABULAR_BOOSTING:Test IDs length: 8
2025-11-18 21:23:58,106 - INFO - Predictions 'catboost' shape: (8,), length: 8
INFO:TABULAR_BOOSTING:Predictions 'catboost' shape: (8,), length: 8
2025-11-18 21:23:58,115 - INFO - Saved predictions: pred_catboost.csv
INFO:TABU

In [63]:
pred_dict

{'catboost': array([42.73166499, 45.94307196, 39.43541628, 40.92841088, 36.26719054,
        41.97345426, 40.59874338, 41.76970097]),
 'lightgbm': None,
 'xgboost': None,
 'blend': array([42.73166499, 45.94307196, 39.43541628, 40.92841088, 36.26719054,
        41.97345426, 40.59874338, 41.76970097])}

In [64]:
# ----- FINAL SUBMISSION -----
logger.info("📝 Preparing final submission file...")

# Найдём итоговый ансамблевый prediction
final_preds = pred_dict.get("catboost")

if final_preds is None:
    logger.warning("Blend predictions are None. Falling back to first available model.")
    for name in ["catboost", "lightgbm", "xgboost"]:
        if pred_dict.get(name) is not None:
            final_preds = pred_dict[name]
            logger.info(f"Using '{name}' predictions for submission.")
            break

if final_preds is None:
    raise ValueError("No predictions available to form submission.")

# Создаём DataFrame с id
submission = pd.DataFrame({id_col: test_ids_series})

# Теперь добавляем target
task_type = CONFIG["task_type"]

if task_type == "multiclass":
    # final_preds — матрица (n, C)
    submission[CONFIG["target_column"]] = np.argmax(final_preds, axis=1)

elif task_type == "binary":
    # финальный prediction — вероятность класса 1
    # если нужен label, заложи порог = 0.5
    submission[CONFIG["target_column"]] = (final_preds >= 0.5).astype(int)

else:  # regression
    submission[CONFIG["target_column"]] = final_preds

# Сохраняем
sub_path = os.path.join(output_dir, "submission.csv")
if "index" in submission.columns:
    submission.drop(columns="index", inplace=True)
submission.to_csv(sub_path, index=False)

logger.info(f"✅ Submission file saved: {sub_path}")
print(submission.head())

2025-11-18 21:24:09,243 - INFO -  Preparing final submission file...
INFO:TABULAR_BOOSTING:📝 Preparing final submission file...
2025-11-18 21:24:09,248 - INFO -  Submission file saved: tabular_boosting_output/submission.csv
INFO:TABULAR_BOOSTING:✅ Submission file saved: tabular_boosting_output/submission.csv


                                 Id  Pawpularity
0  4128bae22183829d2b5fea10effdb0c3    42.731665
1  43a2262d7738e3d420d453815151079e    45.943072
2  4e429cead1848a298432a0acad014c9d    39.435416
3  80bc3ccafcc51b66303c2c263aa38486    40.928411
4  8f49844c382931444e68dffbe20228f4    36.267191


In [65]:
submission.head(35)

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,42.731665
1,43a2262d7738e3d420d453815151079e,45.943072
2,4e429cead1848a298432a0acad014c9d,39.435416
3,80bc3ccafcc51b66303c2c263aa38486,40.928411
4,8f49844c382931444e68dffbe20228f4,36.267191
5,b03f7041962238a7c9d6537e22f9b017,41.973454
6,c978013571258ed6d4637f6e8cc9d6a3,40.598743
7,e0de453c1bffc20c22b072b34b54e50f,41.769701


In [ ]:
# ============================
# Optuna: CPU-only hyperparameter tuning
# ============================
import optuna

def _metric_for_optuna(y_true, y_pred, task_type: str):
    """
    Внутренняя метрика для Optuna: чем БОЛЬШЕ, тем лучше.
    - regression:  -RMSE
    - binary/multiclass: accuracy
    """
    import numpy as np
    from sklearn.metrics import mean_squared_error, accuracy_score

    if task_type == "regression":
        rmse = mean_squared_error(y_true, y_pred, squared=False)
        return -rmse
    else:
        if isinstance(y_pred, np.ndarray) and y_pred.ndim == 2:
            y_hat = np.argmax(y_pred, axis=1)
        else:
            y_hat = y_pred
        acc = accuracy_score(y_true, y_hat)
        return acc


def tune_all_boostings_optuna(
    X,
    y,
    cat_cols,
    CONFIG,
    logger,
    timeout: int = 3600,
    tune_cat: bool = True,
    tune_lgb: bool = True,
    tune_xgb: bool = True,
):
    """
    Подбор гиперов CatBoost, LightGBM и XGBoost на CPU с помощью Optuna.

    tune_cat / tune_lgb / tune_xgb — флаги, какие модели тюнить.
    Например, чтобы тюнить только CatBoost:
        tune_cat=True, tune_lgb=False, tune_xgb=False
    """
    import numpy as np
    from sklearn.model_selection import StratifiedKFold, KFold
    from catboost import CatBoostClassifier, CatBoostRegressor, Pool
    import lightgbm as lgb
    import xgboost as xgb

    task_type = CONFIG.get("task_type", "regression")

    # ====================
    # CV-сплиттер
    # ====================
    n_splits = CONFIG.get("cv_n_splits", 3)
    cv_shuffle = CONFIG.get("cv_shuffle", True)
    cv_random_state = CONFIG.get("cv_random_state", 42)
    cv_stratified = CONFIG.get("cv_stratified", True)

    if task_type == "regression" or not cv_stratified:
        kf = KFold(
            n_splits=n_splits,
            shuffle=cv_shuffle,
            random_state=cv_random_state,
        )
    else:
        kf = StratifiedKFold(
            n_splits=n_splits,
            shuffle=cv_shuffle,
            random_state=cv_random_state,
        )

    # ======================================================
    # 1) CatBoost (CPU, с автоматической настройкой loss/metric)
    # ======================================================
    if tune_cat and CONFIG.get("use_catboost", False):
        logger.info("🔍 Optuna tuning for CatBoost (CPU)...")

        def objective_cat(trial):
            base_params = CONFIG["catboost_params"].copy()

            # принудительно на CPU (Optuna не трогает твоё GPU-обучение)
            base_params["task_type"] = "CPU"
            base_params["thread_count"] = 0
            base_params["verbose"] = False

            # гиперы для тюнинга
            params = {
                **base_params,
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
                "depth": trial.suggest_int("depth", 4, 10),
                "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-2, 10.0, log=True),
                "bagging_temperature": trial.suggest_float("bagging_temperature", 0.0, 10.0),
                "border_count": trial.suggest_int("border_count", 32, 255),
            }

            # ----- loss_function / eval_metric под task_type -----
            if task_type == "regression":
                params["loss_function"] = "MAE"
                params["eval_metric"] = "MAE"
                ModelCls = CatBoostRegressor
            elif task_type == "binary":
                params["loss_function"] = "Logloss"
                params["eval_metric"] = "AUC"
                ModelCls = CatBoostClassifier
            elif task_type == "multiclass":
                params["loss_function"] = "MultiClass"
                params["eval_metric"] = "MultiClass"
                ModelCls = CatBoostClassifier
            else:
                raise ValueError(f"Unsupported task_type '{task_type}' for CatBoost")

            oof_preds = None

            for fold, (tr_idx, va_idx) in enumerate(kf.split(X, y)):
                X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
                y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

                train_pool = Pool(X_tr, y_tr, cat_features=cat_cols)
                valid_pool = Pool(X_va, y_va, cat_features=cat_cols)

                model = ModelCls(**params)

                model.fit(
                    train_pool,
                    eval_set=valid_pool,
                    early_stopping_rounds=CONFIG.get("early_stopping_rounds", 100),
                    verbose=False,
                )

                if task_type == "regression":
                    preds = model.predict(X_va)
                else:
                    proba = model.predict_proba(X_va)
                    if task_type == "binary":
                        preds = proba[:, 1]
                    else:
                        preds = proba

                if oof_preds is None:
                    if isinstance(preds, np.ndarray) and preds.ndim == 2:
                        oof_preds = np.zeros((len(y), preds.shape[1]), dtype=float)
                    else:
                        oof_preds = np.zeros(len(y), dtype=float)
                oof_preds[va_idx] = preds

            score = _metric_for_optuna(y.values, oof_preds, task_type)
            return score

        study_cat = optuna.create_study(direction="maximize")
        study_cat.optimize(objective_cat, timeout=timeout)
        logger.info(f"CatBoost best value: {study_cat.best_value}")
        logger.info(f"CatBoost best params: {study_cat.best_params}")

        CONFIG["catboost_params"].update({
            "learning_rate": study_cat.best_params["learning_rate"],
            "depth": study_cat.best_params["depth"],
            "l2_leaf_reg": study_cat.best_params["l2_leaf_reg"],
            "bagging_temperature": study_cat.best_params["bagging_temperature"],
            "border_count": study_cat.best_params["border_count"],
        })

    # ======================================================
    # 2) LightGBM (CPU, objective/metric по task_type)
    # ======================================================
    if tune_lgb and CONFIG.get("use_lightgbm", False):
        logger.info("🔍 Optuna tuning for LightGBM (CPU)...")

        def objective_lgb(trial):
            base_params = CONFIG["lgb_params"].copy()

            base_params["device_type"] = "cpu"
            base_params["num_threads"] = 0

            if task_type == "regression":
                base_params["objective"] = "regression"
                base_params["metric"] = ["mae"]
            elif task_type == "binary":
                base_params["objective"] = "binary"
                base_params["metric"] = ["auc", "binary_logloss"]
            elif task_type == "multiclass":
                num_classes = len(np.unique(y))
                base_params["objective"] = "multiclass"
                base_params["num_class"] = num_classes
                base_params["metric"] = ["multi_logloss", "multi_error"]
            else:
                raise ValueError(f"Unsupported task_type '{task_type}' for LightGBM")

            params = {
                **base_params,
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
                "num_leaves": trial.suggest_int("num_leaves", 31, 255),
                "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
                "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
                "bagging_freq": trial.suggest_int("bagging_freq", 0, 10),
                "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 100),
                "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 10.0),
                "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 10.0),
            }

            oof_preds = None
            for fold, (tr_idx, va_idx) in enumerate(kf.split(X, y)):
                X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
                y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

                train_set = lgb.Dataset(X_tr, label=y_tr)
                valid_set = lgb.Dataset(X_va, label=y_va)

                model = lgb.train(
                    params,
                    train_set,
                    num_boost_round=5000,
                    valid_sets=[valid_set],
                    valid_names=["valid"],
                    early_stopping_rounds=CONFIG.get("early_stopping_rounds", 100),
                    verbose_eval=False,
                )

                preds = model.predict(X_va, num_iteration=model.best_iteration)

                if oof_preds is None:
                    if isinstance(preds, np.ndarray) and preds.ndim == 2:
                        oof_preds = np.zeros((len(y), preds.shape[1]), dtype=float)
                    else:
                        oof_preds = np.zeros(len(y), dtype=float)
                oof_preds[va_idx] = preds

            score = _metric_for_optuna(y.values, oof_preds, task_type)
            return score

        study_lgb = optuna.create_study(direction="maximize")
        study_lgb.optimize(objective_lgb, timeout=timeout)
        logger.info(f"LightGBM best value: {study_lgb.best_value}")
        logger.info(f"LightGBM best params: {study_lgb.best_params}")

        CONFIG["lgb_params"].update({
            "learning_rate": study_lgb.best_params["learning_rate"],
            "num_leaves": study_lgb.best_params["num_leaves"],
            "feature_fraction": study_lgb.best_params["feature_fraction"],
            "bagging_fraction": study_lgb.best_params["bagging_fraction"],
            "bagging_freq": study_lgb.best_params["bagging_freq"],
            "min_data_in_leaf": study_lgb.best_params["min_data_in_leaf"],
            "lambda_l1": study_lgb.best_params["lambda_l1"],
            "lambda_l2": study_lgb.best_params["lambda_l2"],
        })

    # ======================================================
    # 3) XGBoost (CPU, objective/metric по task_type)
    # ======================================================
    if tune_xgb and CONFIG.get("use_xgboost", False):
        logger.info("🔍 Optuna tuning for XGBoost (CPU)...")

        def objective_xgb(trial):
            base_params = CONFIG["xgb_params"].copy()

            base_params["device"] = "cpu"
            base_params.setdefault("tree_method", "hist")
            base_params.setdefault("nthread", 0)

            if task_type == "regression":
                base_params["objective"] = "reg:squarederror"
                base_params["eval_metric"] = "mae"
            elif task_type == "binary":
                base_params["objective"] = "binary:logistic"
                base_params["eval_metric"] = "auc"
            elif task_type == "multiclass":
                num_classes = len(np.unique(y))
                base_params["objective"] = "multi:softprob"
                base_params["num_class"] = num_classes
                base_params["eval_metric"] = "mlogloss"
            else:
                raise ValueError(f"Unsupported task_type '{task_type}' for XGBoost")

            params = {
                **base_params,
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
                "max_depth": trial.suggest_int("max_depth", 3, 10),
                "min_child_weight": trial.suggest_float("min_child_weight", 1e-2, 10.0, log=True),
                "subsample": trial.suggest_float("subsample", 0.6, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
                "lambda": trial.suggest_float("lambda", 0.0, 10.0),
                "alpha": trial.suggest_float("alpha", 0.0, 10.0),
            }

            oof_preds = None
            for fold, (tr_idx, va_idx) in enumerate(kf.split(X, y)):
                X_tr, X_va = X.iloc[tr_idx], X.iloc[va_idx]
                y_tr, y_va = y.iloc[tr_idx], y.iloc[va_idx]

                dtrain = xgb.DMatrix(X_tr, label=y_tr)
                dvalid = xgb.DMatrix(X_va, label=y_va)

                model = xgb.train(
                    params=params,
                    dtrain=dtrain,
                    num_boost_round=4000,
                    evals=[(dvalid, "valid")],
                    early_stopping_rounds=CONFIG.get("early_stopping_rounds", 100),
                    verbose_eval=False,
                )

                preds = model.predict(dvalid, iteration_range=(0, model.best_iteration + 1))

                if oof_preds is None:
                    if isinstance(preds, np.ndarray) and preds.ndim == 2:
                        oof_preds = np.zeros((len(y), preds.shape[1]), dtype=float)
                    else:
                        oof_preds = np.zeros(len(y), dtype=float)
                oof_preds[va_idx] = preds

            score = _metric_for_optuna(y.values, oof_preds, task_type)
            return score

        study_xgb = optuna.create_study(direction="maximize")
        study_xgb.optimize(objective_xgb, timeout=timeout)
        logger.info(f"XGBoost best value: {study_xgb.best_value}")
        logger.info(f"XGBoost best params: {study_xgb.best_params}")

        CONFIG["xgb_params"].update({
            "learning_rate": study_xgb.best_params["learning_rate"],
            "max_depth": study_xgb.best_params["max_depth"],
            "min_child_weight": study_xgb.best_params["min_child_weight"],
            "subsample": study_xgb.best_params["subsample"],
            "colsample_bytree": study_xgb.best_params["colsample_bytree"],
            "lambda": study_xgb.best_params["lambda"],
            "alpha": study_xgb.best_params["alpha"],
        })

    logger.info("✅ Optuna tuning finished. CONFIG обновлён лучшими гиперпараметрами.")
    return CONFIG

# Пример вызова ТОЛЬКО для CatBoost:
#CONFIG = tune_all_boostings_optuna(X, y, cat_cols, CONFIG, logger, timeout=3600,tune_cat=True, tune_lgb=False, tune_xgb=False)
